In [19]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [20]:
!pip3 install --upgrade pip

In [21]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [22]:
import requests
import pandas as pd
from datetime import datetime
import pytz
import time

# API endpoint and parameters
WEATHER_ENDPOINT = "https://api.openweathermap.org/data/2.5/weather"
POLLUTION_ENDPOINT = "http://api.openweathermap.org/data/2.5/air_pollution"
API_KEY = "5364c78eab167c6b826b39227030cfa4"  # Replace with your actual API key


districts = {
    "Mueang Chon Buri": {"province": "Chon Buri","lat": 13.3622,"lon": 100.98345},
    "Ban Bueng":{"province": "Chon Buri","lat": 13.311,"lon": 101.11214},
    "Nong Yai":{"province": "Chon Buri","lat": 13.11772,"lon": 101.37471},
    "Bang Lamung":{"province": "Chon Buri","lat": 13.04704,"lon": 100.92891},
    "Phan Thong":{"province": "Chon Buri","lat": 13.46804,"lon": 101.09532},
    "Phanat Nikhom":{"province": "Chon Buri","lat": 13.44581,"lon": 101.18445},
    "Si Racha":{"province": "Chon Buri","lat": 13.17372,"lon": 100.93111},
    "Ko Si Chang":{"province": "Chon Buri","lat": 13.15286,"lon": 100.81156},
    "Sattahip":{"province": "Chon Buri","lat": 12.66644,"lon": 100.90073},
    "Bo Thong":{"province": "Chon Buri","lat": 13.24227,"lon": 101.50864},
    "Ko Chan":{"province": "Chon Buri","lat": 13.40253,"lon": 101.388},
    "Mueang Rayong":{"province": "Rayong","lat": 12.71864,"lon": 101.34491},
    "Ban Chang":{"province": "Rayong","lat": 12.72543,"lon": 101.05531},
    "Klaeng":{"province": "Rayong","lat": 12.77179,"lon": 101.65666},
    "Wang Chan":{"province": "Rayong","lat": 12.93646,"lon": 101.52303},
    "Ban Khai":{"province": "Rayong","lat": 12.83827,"lon": 101.3421},
    "Pluak Daeng":{"province": "Rayong", "lat": 12.98333,"lon": 101.25},
    "Khao Chamao":{"province": "Rayong","lat": 12.975,"lon": 101.685},
    "Nikhom Phatthana":{"province": "Rayong","lat": 14.90917,"lon": 101.02673},
    "Mueang Chanthaburi":{"province": "Chanthaburi","lat": 12.60961,"lon": 102.10447},
    "Khlung":{"province": "Chanthaburi","lat": 12.57246,"lon": 102.30783},
    "Tha Mai":{"province": "Chanthaburi","lat": 12.62137,"lon": 102.00481},
    "Pong Nam Ron":{"province": "Chanthaburi","lat": 12.93467,"lon": 102.37189},
    "Makham":{"province": "Chanthaburi","lat": 12.73333,"lon": 102.21667},
    "Laem Sing":{"province": "Chanthaburi","lat": 12.48164,"lon": 102.07375},
    "Soi Dao":{"province": "Chanthaburi","lat": 13.17034,"lon": 102.22796},
    "Kaeng Hang Maeo":{"province": "Chanthaburi", "lat": 13.0969, "lon": 101.88543},
    "Na Yai Am":{"province": "Chanthaburi", "lat": 12.73833, "lon": 101.87502},
    "Khao Khitchakut":{"province": "Chanthaburi", "lat": 12.93726, "lon": 102.09762},
    "Mueang Trat":{"province": "Tart", "lat": 12.26667, "lon": 102.60945},
    "Khlong Yai":{"province": "Tart", "lat": 11.8554, "lon": 102.82596},
    "Khao Saming":{"province": "Tart", "lat": 12.42885, "lon": 102.41784},
    "Bo Rai":{"province": "Tart", "lat": 12.57283, "lon": 102.53714},
    "Laem Ngop":{"province": "Tart","lat": 12.22028, "lon": 102.36558},
    "Ko Kut":{"province": "Tart", "lat": 11.65714, "lon": 102.56694},
    "Ko Chang":{"province": "Tart", "lat": 12.00171, "lon": 102.37267},
    "Mueang Chachoengsao":{"province": "Chachoengsao", "lat": 13.71758, "lon": 101.02152},
    "Bang Khla":{"province": "Chachoengsao", "lat": 13.72144, "lon": 101.20814},
    "Bang Nam Priao":{"province": "Chachoengsao", "lat": 13.84739, "lon": 101.05306},
    "Bang Pakong":{"province": "Chachoengsao","lat": 13.53241, "lon": 100.96045},
    "Ban Pho":{"province": "Chachoengsao","lat": 13.61, "lon": 101.07838},
    "Phanom Sarakham":{"province": "Chachoengsao", "lat": 13.73806, "lon": 101.41202},
    "Ratchasan":{"province": "Chachoengsao", "lat": 13.7866, "lon": 101.28136},
    "Sanam Chai Khet":{"province": "Chachoengsao","lat": 13.62612, "lon": 101.65475},
    "Plaeng Yao":{"province": "Chachoengsao", "lat": 13.56381, "lon": 101.33972},
    "Tha Takiap":{"province": "Chachoengsao","lat": 13.3951,"lon": 101.73457},
    "Khlong Khuean":{"province": "Chachoengsao","lat": 13.79131,"lon": 101.16353},
    "Mueang Prachin Buri":{"province": "Prachin Buri", "lat": 14.04992, "lon": 101.36864},
    "Kabin Buri":{"province": "Prachin Buri", "lat": 13.95114, "lon": 101.71769},
    "Na Di":{"province": "Prachin Buri", "lat": 14.11517, "lon": 101.78189},
    "Ban Sang":{"province": "Prachin Buri", "lat": 13.95005, "lon": 101.25063},
    "Prachantakham":{"province": "Prachin Buri", "lat": 14.22813,  "lon": 101.5744},
    "Si Maha Phot":{"province": "Prachin Buri", "lat": 13.89041, "lon": 101.54181},
    "Si Mahosot": {"province": "Prachin Buri", "lat": 13.87669, "lon": 101.42499},
    "Mueang Nakhon Nayok": {"province": "Nakhon Nayok", "lat": 14.2773, "lon": 101.23379},
    "Pak Phli": {"province": "Nakhon Nayok", "lat": 14.22499, "lon": 101.34482},
    "Ban Na": {"province": "Nakhon Nayok", "lat": 14.2592, "lon": 101.0683},
    "Ongkharak": {"province": "Nakhon Nayok", "lat": 14.07554, "lon": 101.00891},
    "Mueang Sa Kaeo": {"province": "Sa Kaeo", "lat": 13.8240146, "lon": 102.0645933},
    "Khlong Hat": {"province": "Sa Kaeo", "lat": 13.47396, "lon": 102.27604},
    "Ta Phraya": {"province": "Sa Kaeo", "lat": 14.005, "lon": 102.8072},
    "Wang Nam Yen": {"province": "Sa Kaeo", "lat": 13.50325, "lon": 102.18115},
    "Watthana Nakhon": {"province": "Sa Kaeo", "lat": 13.88582, "lon": 102.35652},
    "Aranyaprathet": {"province": "Sa Kaeo", "lat": 13.69276, "lon": 102.50128},
    "Khao Chakan": {"province": "Sa Kaeo", "lat": 13.62293, "lon": 102.02665},
    "Khok Sung": {"province": "Sa Kaeo", "lat": 13.87388, "lon": 102.66075},
    "Wang Sombun": {"province": "Sa Kaeo", "lat": 13.36112, "lon": 102.1249},
    "Mueang Tak": {"province": "Tak", "lat": 16.86968, "lon": 99.12898},
    "Ban Tak": {"province": "Tak", "lat": 17.04325, "lon": 99.08128},
    "Sam Ngao": {"province": "Tak", "lat": 17.24331, "lon": 99.02256},
        "Mae Ramat": {"province": "Tak", "lat": 16.966667, "lon": 98.516667},
    "Tha Song Yang": {"province": "Tak", "lat": 17.226667, "lon": 98.225},
    "Mae Sot": {"province": "Tak", "lat": 16.71667, "lon": 98.56667},
    "Phop Phra": {"province": "Tak", "lat": 16.40, "lon": 98.717},
    "Umphang": {"province": "Tak", "lat": 16.016667, "lon": 98.862778},
    "Wang Chao": {"province": "Tak", "lat": 16.63311, "lon": 99.14826},
    "Mueang Kanchanaburi": {"province": "Kanchanaburi", "lat": 14.06987, "lon": 99.32769},
    "Sai Yok": {"province": "Kanchanaburi", "lat": 14.25652, "lon": 98.93372},
    "Bo Phloi": {"province": "Kanchanaburi", "lat": 14.32517, "lon": 99.51467},
    "Si Sawat": {"province": "Kanchanaburi", "lat": 14.689, "lon": 99.028},
    "Tha Maka": {"province": "Kanchanaburi", "lat": 13.9, "lon": 99.76667},
    "Tha Muang": {"province": "Kanchanaburi", "lat": 13.96118, "lon": 99.64122},
    "Thong Pha Phum": {"province": "Kanchanaburi", "lat": 14.83277, "lon": 98.69233},
    "Sangkhla Buri": {"province": "Kanchanaburi", "lat": 15.15553, "lon": 98.45361},
    "Phanom Thuan": {"province": "Kanchanaburi", "lat": 14.13031, "lon": 99.69858},
    "Lao Khwan": {"province": "Kanchanaburi", "lat": 14.65, "lon": 99.8},
    "Dan Makham Tia": {"province": "Kanchanaburi", "lat": 13.85, "lon": 99.417},
    "Nong Prue": {"province": "Kanchanaburi", "lat": 14.67801, "lon": 99.41075},
    "Huai Krachao": {"province": "Kanchanaburi", "lat": 14.34056, "lon": 99.67633},
    "Mueang Ratchaburi": {"province": "Ratchaburi", "lat": 13.53671, "lon": 99.81712},
    "Chom Bueng": {"province": "Ratchaburi", "lat": 13.62, "lon": 99.59169},
    "Suan Phueng": {"province": "Ratchaburi", "lat": 13.56361, "lon": 99.31133},
    "Damnoen Saduak": {"province": "Ratchaburi", "lat": 13.51825, "lon": 99.95469},
    "Ban Pong": {"province": "Ratchaburi", "lat": 13.8166, "lon": 99.82147},
    "Bang Phae": {"province": "Ratchaburi", "lat": 13.69157, "lon": 99.92982},
    "Photharam": {"province": "Ratchaburi", "lat": 13.683333, "lon": 99.85},
    "Pak Tho": {"province": "Ratchaburi", "lat": 13.38761, "lon": 99.677},
    "Wat Phleng": {"province": "Ratchaburi", "lat": 13.44705, "lon": 99.86959},
    "Ban Kha": {"province": "Ratchaburi", "lat": 13.32374, "lon": 99.37323},
    "Mueang Phetchaburi": {"province": "Phetchaburi", "lat": 13.08173, "lon": 99.96721},
    "Khao Yoi": {"province": "Phetchaburi", "lat": 13.24025, "lon": 99.82428},
    "Nong Ya Plong": {"province": "Phetchaburi", "lat": 13.1259, "lon": 99.4688},
    "Cha-am": {"province": "Phetchaburi", "lat": 12.8, "lon": 99.96667},
    "Tha Yang": {"province": "Phetchaburi", "lat": 12.95772, "lon": 99.90555},
    "Ban Lat": {"province": "Phetchaburi", "lat": 13.04533, "lon": 99.83788},
    "Ban Laem": {"province": "Phetchaburi", "lat": 13.16968, "lon": 99.98771},
    "Kaeng Krachan": {"province": "Phetchaburi", "lat": 12.85919, "lon": 99.46339},
    "Mueang Prachuap Khiri Khan": {"province": "Prachuap Khiri Khan", "lat": 11.82098, "lon": 99.7841},
    "Kui Buri": {"province": "Prachuap Khiri Khan", "lat": 12.08283, "lon": 99.85431},
    "Thap Sakae": {"province": "Prachuap Khiri Khan", "lat": 11.54787, "lon": 99.57159},
    "Bang Saphan": {"province": "Prachuap Khiri Khan", "lat": 11.21422474, "lon": 99.51183651},
    "Bang Saphan Noi": {"province": "Prachuap Khiri Khan", "lat": 11.07057, "lon": 99.34105},
    "Pran Buri": {"province": "Prachuap Khiri Khan", "lat": 12.38487, "lon": 99.90157},
    "Hua Hin": {"province": "Prachuap Khiri Khan", "lat": 12.57065, "lon": 99.95876},
    "Sam Roi Yot": {"province": "Prachuap Khiri Khan", "lat": 12.27081, "lon": 99.87203},
    "Mueang Surat Thani": {"province": "Surat Thani", "lat": 9.136389, "lon": 99.320278},
    "Kanchanadit": {"province": "Surat Thani", "lat": 9.165556, "lon": 99.471111},
    "Don Sak": {"province": "Surat Thani", "lat": 9.313889, "lon": 99.690278},
    "Ko Samui": {"province": "Surat Thani", "lat": 9.58222, "lon": 99.98404},
    "Ko Pha-ngan": {"province": "Surat Thani", "lat": 9.701103979, "lon": 100.022449},
    "Chaiya": {"province": "Surat Thani", "lat": 9.38458298, "lon": 99.18420598},
    "Tha Chana": {"province": "Surat Thani", "lat": 9.572222, "lon": 99.165},
    "Khiri Rat Nikhom": {"province": "Surat Thani","lat": 9.032743276,"lon": 98.95551501},
    "Ban Ta Khun": {"province": "Surat Thani", "lat": 8.999999972, "lon": 98.87},
    "Phanom": {"province": "Surat Thani", "lat": 8.855, "lon": 98.813333},
    "Tha Chang": {"province": "Surat Thani", "lat": 9.268020881, "lon": 99.19492708},
    "Ban Na San": {"province": "Surat Thani", "lat": 8.80837002, "lon": 99.36955296},
    "Ban Na Doem": {"province": "Surat Thani", "lat": 8.935066012, "lon": 99.24618798},
    "Khian Sa": {"province": "Surat Thani", "lat": 8.840833, "lon": 99.194167},
    "Wiang Sa": {"province": "Surat Thani", "lat": 8.654082966, "lon": 99.29299997},
    "Phrasaeng": {"province": "Surat Thani", "lat": 8.568333, "lon": 99.248333},
    "Phunphin": {"province": "Surat Thani", "lat": 9.109722, "lon": 99.231944},
    "Chai Buri": {"province": "Surat Thani", "lat": 8.462778, "lon": 99.076667},
    "Vibhavadi": {"province": "Surat Thani", "lat": 9.238889, "lon": 98.978889},
    "Mueang Chumphon": {"province": "Chumphon", "lat": 10.493056, "lon": 99.180278},
    "Tha Sae": {"province": "Chumphon", "lat": 10.665, "lon": 99.173333},
    "Pathio": {"province": "Chumphon", "lat": 10.708333, "lon": 99.318333},
    "Lang Suan": {"province": "Chumphon", "lat": 9.945, "lon": 99.078333},
    "Lamae": {"province": "Chumphon", "lat": 9.768333, "lon": 99.091667},
    "Phato": {"province": "Chumphon", "lat": 9.791667, "lon": 98.776667},
    "Sawi": {"province": "Chumphon", "lat": 10.230116, "lon": 99.091594},
    "Mueang Ranong": {"province": "Ranong", "lat": 9.972360054, "lon": 98.63969818},
    "La-un": {"province": "Ranong", "lat": 10.115, "lon": 98.75},
    "Kapoe": {"province": "Ranong", "lat": 9.585278, "lon": 98.596111},
    "Kra Buri": {"province": "Ranong", "lat": 10.431667, "lon": 98.788333},
    "Suk Samran": {"province": "Ranong", "lat": 9.345, "lon": 98.43},
    "Mueang Phangnga": {"province": "Phangnga", "lat": 8.443678989, "lon": 98.52025401},
    "Ko Yao": {"province": "Phangnga", "lat": 8.111667, "lon": 98.590833},
    "Kapong": {"province": "Phangnga", "lat": 8.696667, "lon": 98.408333},
    "Takua Thung": {"province": "Phangnga", "lat": 8.392222, "lon": 98.451944},
    "Takua Pa": {"province": "Phangnga", "lat": 8.869444, "lon": 98.343056},
    "Khura Buri": {"province": "Phangnga", "lat": 9.193889, "lon": 98.415},
    "Thap Put": {"province": "Phangnga", "lat": 8.516111, "lon": 98.639722},
    "Thai Mueang": {"province": "Phangnga", "lat": 8.512338043, "lon": 98.28506604},
    "Mueang Krabi": {"province": "Krabi", "lat": 7.776940908, "lon": 98.76289847},
    "Khao Phanom": {"province": "Krabi", "lat": 8.264722, "lon": 99.049167},
    "Ko Lanta": {"province": "Krabi", "lat": 7.530737, "lon": 99.093957},
    "Khlong Thom": {"province": "Krabi", "lat": 7.953333, "lon": 99.145},
    "Ao Luek": {"province": "Krabi", "lat": 8.3775, "lon": 98.721667},
    "Plai Phraya": {"province": "Krabi", "lat": 8.533333, "lon": 98.862778},
    "Lam Thap": {"province": "Krabi", "lat": 8.071667, "lon": 99.291667},
    "Nuea Khlong": {"province": "Krabi", "lat": 8.074722, "lon": 99.003611},
    "Mueang Yala": {"province": "Yala", "lat": 6.546370903, "lon": 101.2843219},
    "Betong": {"province": "Yala", "lat": 5.86121, "lon": 101.22803},
    "Bannang Sata": {"province": "Yala", "lat": 6.25671, "lon": 101.27747},
    "Than To": {"province": "Yala", "lat": 6.08093, "lon": 101.25549},
    "Yaha": {"province": "Yala", "lat": 6.40228, "lon": 101.11542},
    "Raman": {"province": "Yala", "lat": 6.49017, "lon": 101.43402},
    "Kabang": {"province": "Yala", "lat": 6.36658, "lon": 100.97534},
    "Krong Pinang": {"province": "Yala", "lat": 6.39954, "lon": 101.25549},
    "Mueang Narathiwat": {"province": "Narathiwat", "lat": 6.42639, "lon": 101.82308},
    "Tak Bai": {"province": "Narathiwat", "lat": 6.25947, "lon": 102.05461},
    "Bacho": {"province": "Narathiwat", "lat": 6.617, "lon": 101.65},
    "Yi-ngo": {"province": "Narathiwat", "lat": 6.41655, "lon": 101.70035},
    "Ra-ngae": {"province": "Narathiwat", "lat": 6.25504, "lon": 101.70496},
    "Rueso": {"province": "Narathiwat", "lat": 6.3745, "lon": 101.51491},
    "Si Sakhon": {"province": "Narathiwat", "lat": 6.1936, "lon": 101.51257},
    "Waeng": {"province": "Narathiwat", "lat": 5.90042, "lon": 101.86195},
    "Sukhirin": {"province": "Narathiwat", "lat": 5.91513, "lon": 101.73811},
    "Su-ngai Kolok": {"province": "Narathiwat", "lat": 6.0333332, "lon": 101.9666628},
    "Su-ngai Padi": {"province": "Narathiwat", "lat": 6.085833, "lon": 101.881389},
    "Chanae": {"province": "Narathiwat", "lat": 6.111, "lon": 101.69},
    "Cho-airong": {"province": "Narathiwat", "lat": 6.23011, "lon": 101.84721},
    "Mueang Nakhon Si Thammarat": {"province": "Nakhon Si Thammarat", "lat": 8.42, "lon": 99.963333},
    "Phrom Khiri": {"province": "Nakhon Si Thammarat", "lat": 8.521667, "lon": 99.825},
    "Lan Saka": {"province": "Nakhon Si Thammarat", "lat": 8.371667, "lon": 99.805},
    "Chawang": {"province": "Nakhon Si Thammarat", "lat": 8.426111, "lon": 99.504722},
    "Phipun": {"province": "Nakhon Si Thammarat", "lat": 8.56, "lon": 99.608333},
    "Chian Yai": {"province": "Nakhon Si Thammarat", "lat": 8.168333, "lon": 100.145},
    "Cha-uat": {"province": "Nakhon Si Thammarat", "lat": 7.965, "lon": 99.998333},
    "Tha Sala": {"province": "Nakhon Si Thammarat", "lat": 8.666667, "lon": 99.931667},
    "Thung Song": {"province": "Nakhon Si Thammarat", "lat": 8.1586515, "lon": 99.6739885},
    "Na Bon": {"province": "Nakhon Si Thammarat", "lat": 8.261667, "lon": 99.595},
    "Thung Yai": {"province": "Nakhon Si Thammarat", "lat": 8.296667, "lon": 99.375},
    "Pak Phanang": {"province": "Nakhon Si Thammarat", "lat": 8.358333, "lon": 100.206667},
    "Ron Phibun": {"province": "Nakhon Si Thammarat", "lat": 8.18, "lon": 99.855},
    "Sichon": {"province": "Nakhon Si Thammarat", "lat": 9.006667, "lon": 99.901667},
    "Khanom": {"province": "Nakhon Si Thammarat", "lat": 9.204722, "lon": 99.861111},
    "Hua Sai": {"province": "Nakhon Si Thammarat", "lat": 8.04, "lon": 100.275},
    "Bang Khan": {"province": "Nakhon Si Thammarat", "lat": 8.026667, "lon": 99.473333},
    "Tham Phannara": {"province": "Nakhon Si Thammarat", "lat": 8.42, "lon": 99.395},
    "Chulabhorn": {"province": "Nakhon Si Thammarat", "lat": 8.075, "lon": 99.87},
    "Phra Phrom": {"province": "Nakhon Si Thammarat", "lat": 8.338333, "lon": 99.901667},
    "Nopphitam": {"province": "Nakhon Si Thammarat", "lat": 8.678462335, "lon": 99.82972193},
    "Chang Klang": {"province": "Nakhon Si Thammarat", "lat": 8.374167, "lon": 99.568056},
    "Mueang Phatthalung": {"province": "Phatthalung", "lat": 7.61786, "lon": 100.07792},
    "Kong Ra": {"province": "Phatthalung", "lat": 7.467, "lon": 99.95},
    "Khao Chaison": {"province": "Phatthalung", "lat": 7.46305, "lon": 100.09055},
    "Tamot": {"province": "Phatthalung", "lat": 7.33922, "lon": 100.11167},
    "Khuan Khanun": {"province": "Phatthalung", "lat": 7.733, "lon": 100.017},
    "Pak Phayun": {"province": "Phatthalung", "lat": 7.29622, "lon": 100.29542},
    "Si Banphot": {"province": "Phatthalung", "lat": 7.69259, "lon": 99.87322},
    "Pa Bon": {"province": "Phatthalung", "lat": 7.21994, "lon": 100.13078},
    "Bang Kaeo": {"province": "Phatthalung", "lat": 7.41775, "lon": 100.17164},
    "Pa Phayom": {"province": "Phatthalung", "lat": 7.83154, "lon": 99.87448},
    "Srinagarindra": {"province": "Phatthalung", "lat": 7.57574, "lon": 99.91053},
    "Mueang Pattani": { "province": "Pattani", "lat": 6.86814, "lon": 101.25009 },
    "Khok Pho": { "province": "Pattani", "lat": 6.717, "lon": 101.1 },
    "Nong Chik": { "province": "Pattani", "lat": 6.84356, "lon": 101.17803 },
    "Panare": { "province": "Pattani", "lat": 6.86, "lon": 101.49 },
    "Mayo": { "province": "Pattani", "lat": 6.683, "lon": 101.35 },
    "Thung Yang Daeng": { "province": "Pattani", "lat": 6.6416, "lon": 101.44652 },
    "Sai Buri": { "province": "Pattani", "lat": 6.70131, "lon": 101.61675 },
    "Mai Kaen": { "province": "Pattani", "lat": 6.61603, "lon": 101.67864 },
    "Yaring": { "province": "Pattani", "lat": 6.861667, "lon": 101.365 },
    "Yarang": { "province": "Pattani", "lat": 6.6924, "lon": 101.31408 },
    "Kapho": { "province": "Pattani", "lat": 6.60264, "lon": 101.54327 },
    "Mae Lan": { "province": "Pattani", "lat": 6.66833, "lon": 101.23149 },
    "Mueang Phatthalung": { "province": "Phatthalung", "lat": 7.61786, "lon": 100.07792 },
    "Kong Ra": { "province": "Phatthalung", "lat": 7.467, "lon": 99.95 },
    "Khao Chaison": { "province": "Phatthalung", "lat": 7.46305, "lon": 100.09055 },
    "Tamot": { "province": "Phatthalung", "lat": 7.33922, "lon": 100.11167 },
    "Khuan Khanun": { "province": "Phatthalung", "lat": 7.733, "lon": 100.017 },
    "Pak Phayun": { "province": "Phatthalung", "lat": 7.29622, "lon": 100.29542 },
    "Si Banphot": { "province": "Phatthalung", "lat": 7.69259, "lon": 99.87322 },
    "Pa Bon": { "province": "Phatthalung", "lat": 7.21994, "lon": 100.13078 },
    "Bang Kaeo": { "province": "Phatthalung", "lat": 7.41775, "lon": 100.17164 },
    "Pa Phayom": { "province": "Phatthalung", "lat": 7.83154, "lon": 99.87448 },
    "Srinagarindra": { "province": "Phatthalung", "lat": 7.57574, "lon": 99.91053 },
    "Mueang Phangnga": { "province": "Phang Nga", "lat": 8.45091, "lon": 98.52985 },
    "Ko Yao": { "province": "Phang Nga", "lat": 8.117, "lon": 98.6 },
    "Kapong": { "province": "Phang Nga", "lat": 8.74139, "lon": 98.47542 },
    "Takua Thung": { "province": "Phang Nga", "lat": 8.28433, "lon": 98.3895 },
    "Takua Pa": { "province": "Phang Nga", "lat": 8.86883, "lon": 98.33892 },
    "Khura Buri": { "province": "Phang Nga", "lat": 9.18972, "lon": 98.40806 },
    "Thap Put": { "province": "Phang Nga", "lat": 8.53768, "lon": 98.63208 },
    "Thai Mueang": { "province": "Phang Nga", "lat": 8.48995, "lon": 98.31292 },
    "Mueang Songkhla": { "province": "Songkhla", "lat": 7.19882, "lon": 100.5951 },
    "Sathing Phra": { "province": "Songkhla", "lat": 7.467, "lon": 100.433 },
    "Chana": { "province": "Songkhla", "lat": 6.89584, "lon": 100.70126 },
    "Na Thawi": { "province": "Songkhla", "lat": 6.63849, "lon": 100.68173 },
    "Thepha": { "province": "Songkhla", "lat": 6.79726, "lon": 100.91025 },
    "Saba Yoi": { "province": "Songkhla", "lat": 6.617, "lon": 100.95 },
    "Ranot": { "province": "Songkhla", "lat": 7.77768, "lon": 100.32134 },
    "Krasae Sin": { "province": "Songkhla", "lat": 7.61254, "lon": 100.32827 },
    "Rattaphum": { "province": "Songkhla", "lat": 7.07488, "lon": 100.19515 },
    "Sadao": { "province": "Songkhla", "lat": 6.63883, "lon": 100.42342 },
    "Hat Yai": { "province": "Songkhla", "lat": 7.00836, "lon": 100.47668 },
    "Na Mom": { "province": "Songkhla", "lat": 6.9586209, "lon": 100.5352367 },
    "Khuan Niang": { "province": "Songkhla", "lat": 7.18231, "lon": 100.37403 },
    "Bang Klam": { "province": "Songkhla", "lat": 7.06404, "lon": 100.40664 },
    "Singhanakhon": { "province": "Songkhla", "lat": 7.28453, "lon": 100.48809 },
    "Khlong Hoi Khong": { "province": "Songkhla", "lat": 6.86623, "lon": 100.35353 },
    "Mueang Satun": { "province": "Satun", "lat": 6.62314, "lon": 100.06676 },
    "Khuan Don": { "province": "Satun", "lat": 6.76978, "lon": 100.12216 },
    "Khuan Kalong": { "province": "Satun", "lat": 6.90593, "lon": 100.04178 },
    "Tha Phae": { "province": "Satun", "lat": 6.79542, "lon": 99.91985 },
    "La-ngu": { "province": "Satun", "lat": 6.90392, "lon": 99.79836 },
    "Thung Wa": { "province": "Satun", "lat": 7.04915, "lon": 99.76587 },
    "Manang": { "province": "Satun", "lat": 7.01822, "lon": 99.95399 },
    "Mueang Trang": { "province": "Trang", "lat": 7.55633, "lon": 99.61141 },
    "Kantang": { "province": "Trang", "lat": 7.40542, "lon": 99.51561 },
    "Yan Ta Khao": { "province": "Trang", "lat": 7.4258, "lon": 99.73438 },
    "Palian": { "province": "Trang", "lat": 7.2334, "lon": 99.79491 },
    "Sikao": { "province": "Trang", "lat": 7.57122, "lon": 99.33915 },
    "Huai Yot": { "province": "Trang", "lat": 7.78937, "lon": 99.63469 },
    "Wang Wiset": { "province": "Trang", "lat": 7.76061, "lon": 99.40861 },
    "Na Yong": { "province": "Trang", "lat": 7.56257, "lon": 99.74791 },
    "Ratsada": { "province": "Trang", "lat": 7.93779, "lon": 99.66771 },
    "Hat Samran": { "province": "Trang", "lat": 7.23961, "lon": 99.57848 },
    "Mueang Phuket": { "province": "Phuket", "lat": 7.8741947, "lon": 98.3939158 },
    "Kathu": { "province": "Phuket", "lat": 7.8969151, "lon": 98.3020123 },
    "Thalang": { "province": "Phuket", "lat": 8.0320027, "lon": 98.3334626 },
    "Phra Nakhon": { "province": "Bangkok", "lat": 13.7573, "lon": 100.4951 },
    "Dusit": { "province": "Bangkok", "lat": 13.7819, "lon": 100.518 },
    "Nong Khaem": { "province": "Bangkok", "lat": 13.8563, "lon": 100.863 },
    "Bang Rak": { "province": "Bangkok", "lat": 13.7263, "lon": 100.5279 },
    "Bang Khen": { "province": "Bangkok", "lat": 13.8644, "lon": 100.6146 },
    "Bang Kapi": { "province": "Bangkok", "lat": 13.7628, "lon": 100.6456 },
    "Pathum Wan": { "province": "Bangkok", "lat": 13.7462, "lon": 100.5308 },
    "Pom Prap Sattru Phai": { "province": "Bangkok", "lat": 13.7515, "lon": 100.5108 },
    "Phra Khanong": { "province": "Bangkok", "lat": 13.6903, "lon": 100.6086 },
    "Min Buri": { "province": "Bangkok", "lat": 13.8155, "lon": 100.7353 },
    "Lat Krabang": { "province": "Bangkok", "lat": 13.7506, "lon": 100.7943 },
    "Yan Nawa": { "province": "Bangkok", "lat": 13.6943, "lon": 100.5394 },
    "Samphanthawong": { "province": "Bangkok", "lat": 13.7362, "lon": 100.5136 },
    "Phaya Thai": { "province": "Bangkok", "lat": 13.7889, "lon": 100.5529 },
    "Thon Buri": { "province": "Bangkok", "lat": 13.7078, "lon": 100.4868 },
    "Bangkok Yai": { "province": "Bangkok", "lat": 13.7294, "lon": 100.4741 },
    "Huai Khwang": { "province": "Bangkok", "lat": 13.7699, "lon": 100.5865 },
    "Khlong San": { "province": "Bangkok", "lat": 13.7287, "lon": 100.501 },
    "Taling Chan": { "province": "Bangkok", "lat": 13.781, "lon": 100.4309 },
    "Bangkok Noi": { "province": "Bangkok", "lat": 13.7548, "lon": 100.4694 },
    "Bang Khun Thian": { "province": "Bangkok", "lat": 13.6332, "lon": 100.4267 },
    "Phasi Charoen": { "province": "Bangkok", "lat": 13.724, "lon": 100.4385 },
    "Nong Chok": { "province": "Bangkok", "lat": 13.7054, "lon": 100.3432 },
    "Rat Burana": { "province": "Bangkok", "lat": 13.6715, "lon": 100.5088 },
    "Bang Phlat": { "province": "Bangkok", "lat": 13.7847, "lon": 100.4945 },
    "Din Daeng": { "province": "Bangkok", "lat": 13.7605, "lon": 100.5549 },
    "Bueng Kum": { "province": "Bangkok", "lat": 13.8038, "lon": 100.6475 },
    "Sathon": { "province": "Bangkok", "lat": 13.715, "lon": 100.5335 },
    "Bang Sue": { "province": "Bangkok", "lat": 13.8064, "lon": 100.5308 },
    "Chatuchak": { "province": "Bangkok", "lat": 13.8339, "lon": 100.5756 },
    "Bang Kho Laem": { "province": "Bangkok", "lat": 13.6982, "lon": 100.5048 },
    "Prawet": { "province": "Bangkok", "lat": 13.7, "lon": 100.6755 },
    "Khlong Toei": { "province": "Bangkok", "lat": 13.7189, "lon": 100.5672 },
    "Suan Luang": { "province": "Bangkok", "lat": 13.7204, "lon": 100.6296 },
    "Chom Thong": { "province": "Bangkok", "lat": 13.6991, "lon": 100.4673 },
    "Don Mueang": { "province": "Bangkok", "lat": 13.9272, "lon": 100.5859 },
    "Ratchathewi": { "province": "Bangkok", "lat": 13.758, "lon": 100.5345 },
    "Lat Phrao": { "province": "Bangkok", "lat": 13.8241, "lon": 100.6088 },
      "Vadhana": { "province": "Bangkok", "lat": 13.7308, "lon": 100.5864 },
    "Bang Khae": { "province": "Bangkok", "lat": 13.7098, "lon": 100.3959 },
    "Lak Si": { "province": "Bangkok", "lat": 13.8898, "lon": 100.5696 },
    "Sai Mai": { "province": "Bangkok", "lat": 13.9047, "lon": 100.6572 },
    "Khan Na Yao": { "province": "Bangkok", "lat": 13.8271, "lon": 100.6743 },
    "Saphan Sung": { "province": "Bangkok", "lat": 13.7685, "lon": 100.6918 },
    "Wang Thonglang": { "province": "Bangkok", "lat": 13.7864, "lon": 100.6087 },
    "Khlong Sam Wa": { "province": "Bangkok", "lat": 13.8601, "lon": 100.704 },
    "Bang Na": { "province": "Bangkok", "lat": 13.6639, "lon": 100.6132 },
    "Thawi Watthana": { "province": "Bangkok", "lat": 13.7878, "lon": 100.3638 },
    "Thung Khru": { "province": "Bangkok", "lat": 13.6472, "lon": 100.4958 },
    "Bang Bon": { "province": "Bangkok", "lat": 13.6592, "lon": 100.3991 },
    "Mueang Chai Nat": { "province": "Chai Nat", "lat": 15.2001, "lon": 100.1479 },
    "Manorom": { "province": "Chai Nat", "lat": 15.3389, "lon": 100.1479 },
    "Wat Sing": { "province": "Chai Nat", "lat": 15.2081, "lon": 99.9689 },
    "Sapphaya": { "province": "Chai Nat", "lat": 15.1375, "lon": 100.243056 },
    "Sankhaburi": { "province": "Chai Nat", "lat": 15.0133, "lon": 100.1927 },
    "Hankha": { "province": "Chai Nat", "lat": 15.0694, "lon": 99.9689 },
    "Nong Mamong": { "province": "Chai Nat", "lat": 15.2141, "lon": 99.8348 },
    "Noen Kham": { "province": "Chai Nat", "lat": 14.9829, "lon": 99.8348 },
    "Mueang Nakhon Pathom": { "province": "Nakhon Pathom", "lat": 13.825, "lon": 100.0136 },
    "Kamphaeng Saen": { "province": "Nakhon Pathom", "lat": 14.0341, "lon": 99.9465 },
    "Nakhon Chai Si": { "province": "Nakhon Pathom", "lat": 13.8176, "lon": 100.1927 },
    "Don Tum": { "province": "Nakhon Pathom", "lat": 13.9478, "lon": 100.0919 },
    "Bang Len": { "province": "Nakhon Pathom", "lat": 14.0229, "lon": 100.2152 },
    "Sam Phran": { "province": "Nakhon Pathom", "lat": 13.7242, "lon": 100.2376 },
    "Phutthamonthon": { "province": "Nakhon Pathom", "lat": 13.8248, "lon": 100.2937 },
    "Mueang Kamphaeng Phet": { "province": "Kamphaeng Phet", "lat": 16.46, "lon": 99.5006 },
    "Sai Ngam": { "province": "Kamphaeng Phet", "lat": 16.4435, "lon": 99.8571 },
    "Khlong Lan": { "province": "Kamphaeng Phet", "lat": 16.2854, "lon": 99.2342 },
    "Khanu Woralaksaburi": { "province": "Kamphaeng Phet", "lat": 15.9863, "lon": 99.6787 },
    "Khlong Khlung": { "province": "Kamphaeng Phet", "lat": 16.2412, "lon": 99.701 },
    "Phran Kratai": { "province": "Kamphaeng Phet", "lat": 16.665, "lon": 99.5883 },
    "Lan Krabue": { "province": "Kamphaeng Phet", "lat": 16.6055, "lon": 99.8795 },
    "Sai Thong Watthana": { "province": "Kamphaeng Phet", "lat": 16.3133, "lon": 99.8316 },
    "Pang Sila Thong": { "province": "Kamphaeng Phet", "lat": 16.0244, "lon": 99.3451 },
    "Bueng Samakkhi": { "province": "Kamphaeng Phet", "lat": 16.2287, "lon": 99.9689 },
    "Kosamphi Nakhon": { "province": "Kamphaeng Phet", "lat": 16.5857, "lon": 99.3007 },
    "Mueang Nakhon Sawan": { "province": "Nakhon Sawan", "lat": 15.6873, "lon": 100.1255 },
    "Krok Phra": { "province": "Nakhon Sawan", "lat": 15.5765, "lon": 100.0136 },
    "Chum Saeng": { "province": "Nakhon Sawan", "lat": 15.8811, "lon": 100.3025 },
    "Nong Bua": { "province": "Nakhon Sawan", "lat": 15.865, "lon": 100.586111 },
    "Banphot Phisai": { "province": "Nakhon Sawan", "lat": 15.9741, "lon": 99.9465 },
    "Kao Liao": { "province": "Nakhon Sawan", "lat": 15.8971, "lon": 100.1031 },
    "Takhli": { "province": "Nakhon Sawan", "lat": 15.2823, "lon": 100.3722 },
    "Tha Tako": { "province": "Nakhon Sawan", "lat": 15.6703, "lon": 100.4846 },
    "Phaisali": { "province": "Nakhon Sawan", "lat": 15.569, "lon": 100.6647 },
    "Phayuha Khiri": { "province": "Nakhon Sawan", "lat": 15.4978, "lon": 100.2152 },
    "Lat Yao": { "province": "Nakhon Sawan", "lat": 15.7541, "lon": 99.7905 },
    "Tak Fa": { "province": "Nakhon Sawan", "lat": 15.3705, "lon": 100.4621 },
    "Mae Wong": { "province": "Nakhon Sawan", "lat": 15.8364, "lon": 99.3895 },
    "Mae Poen": { "province": "Nakhon Sawan", "lat": 15.6577, "lon": 99.4691 },
    "Chum Ta Bong": { "province": "Nakhon Sawan", "lat": 15.6316, "lon": 99.5527 },
    "Mueang Nonthaburi": { "province": "Nonthaburi", "lat": 13.862, "lon": 100.4959 },
    "Bang Kruai": { "province": "Nonthaburi", "lat": 13.8136, "lon": 100.4228 },
    "Bang Yai": { "province": "Nonthaburi", "lat": 13.8448, "lon": 100.361 },
    "Bang Bua Thong": { "province": "Nonthaburi", "lat": 13.9355, "lon": 100.3835 },
    "Sai Noi": { "province": "Nonthaburi", "lat": 14.0071, "lon": 100.3161 },
    "Pak Kret": { "province": "Nonthaburi", "lat": 13.9077, "lon": 100.4959 },
    "Mueang Pathum Thani": { "province": "Pathum Thani", "lat": 13.9854, "lon": 100.5521 },
    "Khlong Luang": { "province": "Pathum Thani", "lat": 14.1169, "lon": 100.6872 },
    "Thanyaburi": { "province": "Pathum Thani", "lat": 13.9924, "lon": 100.6534 },
    "Nong Suea": { "province": "Pathum Thani", "lat": 14.1786, "lon": 100.845 },
    "Lat Lum Kaeo": { "province": "Pathum Thani", "lat": 14.0371, "lon": 100.4172 },
    "Lam Luk Ka": { "province": "Pathum Thani", "lat": 13.9755, "lon": 100.7774 },
    "Sam Khok": { "province": "Pathum Thani", "lat": 14.0791, "lon": 100.5071 },
    "Phra Nakhon Si Ayutthaya": { "province": "Phra Nakhon Si Ayutthaya", "lat": 14.3524, "lon": 100.5521 },
    "Tha Ruea": { "province": "Phra Nakhon Si Ayutthaya", "lat": 14.5301, "lon": 100.6872 },
    "Nakhon Luang": { "province": "Phra Nakhon Si Ayutthaya", "lat": 14.4752, "lon": 100.6309 },
    "Bang Sai": { "province": "Phra Nakhon Si Ayutthaya", "lat": 14.2186, "lon": 100.4621 },
    "Bang Ban": { "province": "Phra Nakhon Si Ayutthaya", "lat": 14.4024, "lon": 100.4621 },
    "Bang Pa-in": { "province": "Phra Nakhon Si Ayutthaya", "lat": 14.2586, "lon": 100.5971 },
    "Bang Pahan": { "province": "Phra Nakhon Si Ayutthaya", "lat": 14.4793, "lon": 100.5409 },
    "Phak Hai": { "province": "Phra Nakhon Si Ayutthaya", "lat": 14.4543, "lon": 100.3273 },
    "Phachi": { "province": "Phra Nakhon Si Ayutthaya", "lat": 14.4361, "lon": 100.7323 },
    "Lat Bua Luang": { "province": "Phra Nakhon Si Ayutthaya", "lat": 14.1767, "lon": 100.3722 },
    "Wang Noi": { "province": "Phra Nakhon Si Ayutthaya", "lat": 14.2525, "lon": 100.7323 },
    "Sena": { "province": "Phra Nakhon Si Ayutthaya", "lat": 14.2685, "lon": 100.3722 },
    "Bang Sai": { "province": "Phra Nakhon Si Ayutthaya", "lat": 14.3064, "lon": 100.2937 },
    "Uthai": { "province": "Phra Nakhon Si Ayutthaya", "lat": 14.3463, "lon": 100.6872 },
    "Maha Rat": { "province": "Phra Nakhon Si Ayutthaya", "lat": 14.5822, "lon": 100.5521 },
    "Ban Phraek": { "province": "Phra Nakhon Si Ayutthaya", "lat": 14.6477, "lon": 100.5825 },
    "Mueang Phichit": { "province": "Phichit", "lat": 16.3953, "lon": 100.3722 },
    "Wang Sai Phun": { "province": "Phichit", "lat": 16.3398, "lon": 100.5521 },
    "Pho Prathap Chang": { "province": "Phichit", "lat": 16.3111, "lon": 100.1927 },
    "Taphan Hin": { "province": "Phichit", "lat": 16.2071, "lon": 100.4172 },
    "Bang Mun Nak": { "province": "Phichit", "lat": 16.0214, "lon": 100.4172 },
    "Pho Thale": { "province": "Phichit", "lat": 16.0543, "lon": 100.2152 },
    "Sam Ngam": { "province": "Phichit", "lat": 16.4085, "lon": 100.1031 },
    "Tap Khlo": { "province": "Phichit", "lat": 16.1616, "lon": 100.5966 },
    "Sak Lek": { "province": "Phichit", "lat": 16.51, "lon": 100.47 },
    "Bueng Na Rang": { "province": "Phichit", "lat": 16.1737, "lon": 100.1479 },
    "Dong Charoen": { "province": "Phichit", "lat": 16.0283, "lon": 100.6266 },
    "Wachirabarami": { "province": "Phichit", "lat": 16.5016, "lon": 100.1031 },
    "Mueang Phitsanulok": { "province": "Phitsanulok", "lat": 16.799, "lon": 100.2152 },
    "Nakhon Thai": { "province": "Phitsanulok", "lat": 17.1221, "lon": 100.8395 },
    "Chat Trakan": { "province": "Phitsanulok", "lat": 17.3355, "lon": 100.6647 },
    "Bang Rakam": { "province": "Phitsanulok", "lat": 16.7145, "lon": 100.036 },
    "Bang Krathum": { "province": "Phitsanulok", "lat": 16.5837, "lon": 100.3273 },
    "Phrom Phiram": { "province": "Phitsanulok", "lat": 17.0834, "lon": 100.1255 },
    "Wat Bot": { "province": "Phitsanulok", "lat": 16.9802, "lon": 100.3333 },
    "Wang Thong": { "province": "Phitsanulok", "lat": 16.8739, "lon": 100.5746 },
    "Noen Maprang": { "province": "Phitsanulok", "lat": 16.5899, "lon": 100.6647 },
    "Mueang Phetchabun": { "province": "Phetchabun", "lat": 16.3266, "lon": 101.2524 },
    "Chon Daen": { "province": "Phetchabun", "lat": 16.116, "lon": 100.8451 },
    "Lom Sak": { "province": "Phetchabun", "lat": 16.6981, "lon": 101.2524 },
    "Lom Kao": { "province": "Phetchabun", "lat": 17.0212, "lon": 101.2977 },
    "Wichian Buri": { "province": "Phetchabun", "lat": 15.6572, "lon": 101.1066 },
    "Si Thep": { "province": "Phetchabun", "lat": 15.4495, "lon": 101.207 },
    "Nong Phai": { "province": "Phetchabun", "lat": 16.0093, "lon": 101.1164 },
    "Bueng Sam Phan": { "province": "Phetchabun", "lat": 15.8333, "lon": 100.9355 },
    "Nam Nao": { "province": "Phetchabun", "lat": 16.8198, "lon": 101.5703 },
    "Wang Pong": { "province": "Phetchabun", "lat": 16.3725, "lon": 100.8226 },
    "Khao Kho": { "province": "Phetchabun", "lat": 16.5757, "lon": 100.9355 },
    "Mueang Lop Buri": { "province": "Lop Buri", "lat": 14.8302, "lon": 100.6647 },
    "Phatthana Nikhom": { "province": "Lop Buri", "lat": 14.9052, "lon": 101.0259 },
    "Khok Samrong": { "province": "Lop Buri", "lat": 15.0103, "lon": 100.7549 },
    "Chai Badan": { "province": "Lop Buri", "lat": 15.1772, "lon": 101.1164 },
    "Tha Wung": { "province": "Lop Buri", "lat": 14.8145, "lon": 100.5071 },
    "Ban Mi": { "province": "Lop Buri", "lat": 15.1111, "lon": 100.5746 },
    "Tha Luang": { "province": "Lop Buri", "lat": 15.0586, "lon": 101.1844 },
    "Sa Bot": { "province": "Lop Buri", "lat": 15.2246, "lon": 100.8564 },
    "Khok Charoen": { "province": "Lop Buri", "lat": 15.4676, "lon": 100.8451 },
    "Lam Sonthi": { "province": "Lop Buri", "lat": 15.4074, "lon": 101.3545 },
    "Nong Muang": { "province": "Lop Buri", "lat": 15.2674, "lon": 100.6872 },
    "Mueang Samut Prakan": { "province": "Samut Prakan", "lat": 13.57, "lon": 100.6422 },
    "Bang Bo": { "province": "Samut Prakan", "lat": 13.6058, "lon": 100.8677 },
    "Bang Phli": { "province": "Samut Prakan", "lat": 13.6117, "lon": 100.7323 },
    "Phra Pradaeng": { "province": "Samut Prakan", "lat": 13.6533, "lon": 100.5634 },
    "Phra Samut Chedi": { "province": "Samut Prakan", "lat": 13.5738, "lon": 100.5521 },
    "Bang Sao Thong": { "province": "Samut Prakan", "lat": 13.6425, "lon": 100.8113 },
    "Mueang Samut Songkhram": { "province": "Samut Songkhram", "lat": 13.4133, "lon": 100.0136 },
    "Bang Khonthi": { "province": "Samut Songkhram", "lat": 13.4726, "lon": 99.9577 },
    "Amphawa": { "province": "Samut Songkhram", "lat": 13.3602, "lon": 99.913 },
    "Mueang Samut Sakhon": { "province": "Samut Sakhon", "lat": 13.5413, "lon": 100.2376 },
    "Krathum Baen": { "province": "Samut Sakhon", "lat": 13.6656, "lon": 100.2712 },
    "Ban Phaeo": { "province": "Samut Sakhon", "lat": 13.5905, "lon": 100.1077 },
    "Mueang Saraburi": { "province": "Saraburi", "lat": 14.5196, "lon": 100.9129 },
    "Kaeng Khoi": { "province": "Saraburi", "lat": 14.6292, "lon": 101.0259 },
    "Nong Khae": { "province": "Saraburi", "lat": 14.384, "lon": 100.8677 },
    "Wihan Daeng": { "province": "Saraburi", "lat": 14.3777, "lon": 101.0033 },
    "Nong Saeng": { "province": "Saraburi", "lat": 14.4899, "lon": 100.8113 },
    "Ban Mo": { "province": "Saraburi", "lat": 14.62, "lon": 100.7323 },
    "Don Phut": { "province": "Saraburi", "lat": 14.5922, "lon": 100.6283 },
    "Nong Don": { "province": "Saraburi", "lat": 14.702, "lon": 100.6985 },
    "Phra Phutthabat": { "province": "Saraburi", "lat": 14.7099, "lon": 100.7774 },
    "Sao Hai": { "province": "Saraburi", "lat": 14.5698, "lon": 100.8226 },
    "Muak Lek": { "province": "Saraburi", "lat": 14.824, "lon": 101.2751 },
    "Wang Muang": { "province": "Saraburi", "lat": 14.8306, "lon": 101.1391 },
    "Chaloem Phra Kiat": { "province": "Saraburi", "lat": 14.6575, "lon": 100.9129},
    "Mueang Sing Buri": { "province": "Sing Buri", "lat": 14.8998, "lon": 100.4059 },
    "Bang Rachan": { "province": "Sing Buri", "lat": 14.9169, "lon": 100.2824 },
    "Khai Bang Rachan": { "province": "Sing Buri", "lat": 14.8126, "lon": 100.2937 },
    "Phrom Buri": { "province": "Sing Buri", "lat": 14.7825, "lon": 100.4509 },
    "Tha Chang": { "province": "Sing Buri", "lat": 14.7795, "lon": 100.3891 },
    "In Buri": { "province": "Sing Buri", "lat": 15.0072, "lon": 100.3273 },
    "Mueang Sukhothai": { "province": "Sukhothai", "lat": 17.0298, "lon": 99.7902 },
    "Ban Dan Lan Hoi": { "province": "Sukhothai", "lat": 17.0061, "lon": 99.5747 },
    "Khiri Mat": { "province": "Sukhothai", "lat": 16.8208, "lon": 99.7679 },
    "Kong Krailat": { "province": "Sukhothai", "lat": 16.9525, "lon": 99.9761 },
    "Si Satchanalai": { "province": "Sukhothai", "lat": 17.5732, "lon": 99.6787 },
    "Si Samrong": { "province": "Sukhothai", "lat": 17.1647, "lon": 99.8619 },
    "Sawankhalok": { "province": "Sukhothai", "lat": 17.2836, "lon": 99.8571 },
    "Si Nakhon": { "province": "Sukhothai", "lat": 17.348, "lon": 99.9908 },
    "Thung Saliam": { "province": "Sukhothai", "lat": 17.3679, "lon": 99.5673 },
    "Mueang Suphan Buri": { "province": "Suphan Buri", "lat": 14.47418, "lon": 100.12218 },
    "Doem Bang Nang Buat": { "province": "Suphan Buri", "lat": 14.83333, "lon": 100.1 },
    "Dan Chang": { "province": "Suphan Buri", "lat": 14.88388, "lon": 99.51642 },
    "Bang Pla Ma": { "province": "Suphan Buri", "lat": 14.34985, "lon": 100.14479 },
    "Si Prachan": { "province": "Suphan Buri", "lat": 14.617, "lon": 100.15 },
    "Don Chedi": { "province": "Suphan Buri", "lat": 14.64804, "lon": 99.94734 },
    "Song Phi Nong": { "province": "Suphan Buri", "lat": 14.18895, "lon": 99.9758 },
    "Sam Chuk": { "province": "Suphan Buri", "lat": 14.74219, "lon": 100.09531 },
    "U Thong": { "province": "Suphan Buri", "lat": 14.41093, "lon": 99.88949 },
    "Nong Ya Sai": { "province": "Suphan Buri", "lat": 14.77644, "lon": 99.85751 },
    "Mueang Ang Thong": { "province": "Ang Thong", "lat": 14.58883098, "lon": 100.4543315 },
    "Chaiyo": { "province": "Ang Thong", "lat": 14.6491667, "lon": 100.4786111 },
    "Pa Mok": { "province": "Ang Thong", "lat": 14.4862, "lon": 100.4573 },
    "Pho Thong": { "province": "Ang Thong", "lat": 14.667, "lon": 100.417 },
    "Sawaeng Ha": { "province": "Ang Thong", "lat": 14.74635, "lon": 100.28963 },
    "Wiset Chai Chan": { "province": "Ang Thong", "lat": 14.5945, "lon": 100.33825 },
    "Samko": { "province": "Ang Thong", "lat": 14.60731, "lon": 100.2488 },
    "Mueang Uthai Thani": { "province": "Uthai Thani", "lat": 15.37266518, "lon": 100.0336665 },
    "Thap Than": { "province": "Uthai Thani", "lat": 15.46063, "lon": 99.89166 },
    "Sawang Arom": { "province": "Uthai Thani", "lat": 15.6025, "lon": 99.7849 },
    "Nong Chang": { "province": "Uthai Thani", "lat": 15.392, "lon": 99.844 },
    "Nong Khayang": { "province": "Uthai Thani", "lat": 15.362, "lon": 99.929 },
    "Ban Rai": { "province": "Uthai Thani", "lat": 15.24921, "lon": 99.32817 },
    "Lan Sak": { "province": "Uthai Thani", "lat": 15.52289, "lon": 99.4611 },
    "Huai Khot": { "province": "Uthai Thani", "lat": 15.32761, "lon": 99.51941 },
    "Mueang Nakhon Ratchasima": { "province": "Nakhon Ratchasima", "lat": 14.971111, "lon": 102.099722 },
    "Khon Buri": { "province": "Nakhon Ratchasima", "lat": 14.523333, "lon": 102.248333 },
    "Soeng Sang": { "province": "Nakhon Ratchasima", "lat": 14.4254832, "lon": 102.3962593 },
    "Khong": { "province": "Nakhon Ratchasima", "lat": 15.42230782, "lon": 102.4190103 },
    "Ban Lueam": { "province": "Nakhon Ratchasima", "lat": 15.63704614, "lon": 102.1338802 },
    "Chakkarat": { "province": "Nakhon Ratchasima", "lat": 15.013333, "lon": 102.413333 },
    "Chok Chai": { "province": "Nakhon Ratchasima", "lat": 14.69333841, "lon": 102.1810966 },
    "Dan Khun Thot": { "province": "Nakhon Ratchasima", "lat": 15.20972, "lon": 101.76583 },
    "Non Thai": { "province": "Nakhon Ratchasima", "lat": 15.27430626, "lon": 102.1474181 },
    "Non Sung": { "province": "Nakhon Ratchasima", "lat": 15.20446914, "lon": 102.3114206 },
    "Kham Sakaesaeng": { "province": "Nakhon Ratchasima", "lat": 15.32944, "lon": 102.17083 },
    "Bua Yai": { "province": "Nakhon Ratchasima", "lat": 15.50678002, "lon": 102.3969855 },
    "Prathai": { "province": "Nakhon Ratchasima", "lat": 15.51920941, "lon": 102.7911416 },
    "Pak Thong Chai": { "province": "Nakhon Ratchasima", "lat": 14.72296504, "lon": 102.0255925 },
    "Phimai": { "province": "Nakhon Ratchasima", "lat": 15.220057, "lon": 102.493849 },
    "Huai Thalaeng": { "province": "Nakhon Ratchasima", "lat": 15.0343203, "lon": 102.7396875 },
    "Chum Phuang": { "province": "Nakhon Ratchasima", "lat": 15.348889, "lon": 102.741944 },
    "Sung Noen": { "province": "Nakhon Ratchasima", "lat": 14.91794423, "lon": 101.8098215 },
    "Kham Thale So": { "province": "Nakhon Ratchasima", "lat": 14.96083, "lon": 101.9475 },
    "Sikhio": { "province": "Nakhon Ratchasima", "lat": 14.89167, "lon": 101.72333 },
    "Pak Chong": { "province": "Nakhon Ratchasima", "lat": 14.6552917, "lon": 101.4339148 },
    "Nong Bun Mak": { "province": "Nakhon Ratchasima", "lat": 14.73972, "lon": 102.36583 },
    "Kaeng Sanam Nang": { "province": "Nakhon Ratchasima", "lat": 15.7696853, "lon": 102.2701234 },
    "Non Daeng": { "province": "Nakhon Ratchasima", "lat": 15.41083, "lon": 102.54028 },
    "Wang Nam Khiao": { "province": "Nakhon Ratchasima", "lat": 14.4708894, "lon": 101.8246093 },
    "Thepharak": { "province": "Nakhon Ratchasima", "lat": 15.30944, "lon": 101.55056 },
    "Mueang Yang": { "province": "Nakhon Ratchasima", "lat": 15.42333, "lon": 102.90167 },
    "Phra Thong Kham": { "province": "Nakhon Ratchasima", "lat": 15.30667, "lon": 101.98167 },
    "Lam Thamenchai": { "province": "Nakhon Ratchasima", "lat": 15.353333, "lon": 102.918333 },
    "Bua Lai": { "province": "Nakhon Ratchasima", "lat": 15.66278, "lon": 102.52722 },
    "Sida": { "province": "Nakhon Ratchasima", "lat": 15.5425, "lon": 102.56583 },
    "Mueang Buri Ram": { "province": "Buri Ram", "lat": 14.995, "lon": 103.10333 },
    "Khu Mueang": { "province": "Buri Ram", "lat": 15.27167, "lon": 103.00167 },
    "Krasang": { "province": "Buri Ram", "lat": 14.92167, "lon": 103.30167 },
    "Nang Rong": { "province": "Buri Ram", "lat": 14.62833, "lon": 102.79333 },
    "Nong Ki": { "province": "Buri Ram", "lat": 14.68833, "lon": 102.53333 },
    "Lahan Sai": { "province": "Buri Ram", "lat": 14.41167, "lon": 102.86 },
    "Prakhon Chai": { "province": "Buri Ram", "lat": 14.61030528, "lon": 103.0801044 },
    "Ban Kruat": { "province": "Buri Ram", "lat": 14.3461981, "lon": 103.1631166 },
    "Phutthaisong": { "province": "Buri Ram", "lat": 15.54471764, "lon": 103.0004571 },
    "Lam Plai Mat": { "province": "Buri Ram", "lat": 15.025, "lon": 102.83833 },
    "Satuek": { "province": "Buri Ram", "lat": 15.2975, "lon": 103.29194 },
    "Pakham": { "province": "Buri Ram", "lat": 14.405851, "lon": 102.572167 },
    "Na Pho": { "province": "Buri Ram", "lat": 15.64523306, "lon": 102.95014 },
    "Nong Hong": { "province": "Buri Ram", "lat": 14.85, "lon": 102.688333 },
    "Phlapphla Chai": { "province": "Buri Ram", "lat": 14.698366, "lon": 103.116395 },
    "Huai Rat": { "province": "Buri Ram", "lat": 14.9907736, "lon": 103.1958313 },
    "Non Suwan": { "province": "Buri Ram", "lat": 14.57833, "lon": 102.59833 },
    "Chamni": { "province": "Buri Ram", "lat": 14.78833, "lon": 102.84167 },
    "Ban Mai Chaiyaphot": { "province": "Buri Ram", "lat": 15.530062, "lon": 102.889992 },
    "Non Din Daeng": { "province": "Buri Ram", "lat": 14.315, "lon": 102.74333 },
    "Ban Dan": { "province": "Buri Ram", "lat": 15.10722, "lon": 103.17472 },
    "Khaen Dong": { "province": "Buri Ram", "lat": 15.317225, "lon": 103.156571},
    "Mueang Surin": { "province": "Surin", "lat": 14.88433, "lon": 103.490522 },
    "Chumphon Buri": { "province": "Surin", "lat": 15.35083, "lon": 103.38889 },
    "Tha Tum": { "province": "Surin", "lat": 15.31833, "lon": 103.675 },
    "Chom Phra": { "province": "Surin", "lat": 15.1175, "lon": 103.61139 },
    "Prasat": { "province": "Surin", "lat": 14.64056, "lon": 103.40472 },
    "Kap Choeng": { "province": "Surin", "lat": 14.47472, "lon": 103.59417 },
    "Rattanaburi": { "province": "Surin", "lat": 15.32028, "lon": 103.85694 },
    "Sanom": { "province": "Surin", "lat": 15.19889, "lon": 103.76167 },
    "Sikhoraphum": { "province": "Surin", "lat": 14.94396316, "lon": 103.8038376 },
    "Sangkha": { "province": "Surin", "lat": 14.6161736, "lon": 103.8846545 },
    "Lamduan": { "province": "Surin", "lat": 14.718503, "lon": 103.667466 },
    "Samrong Thap": { "province": "Surin", "lat": 15.0225, "lon": 103.93611 },
    "Buachet": { "province": "Surin", "lat": 14.52667, "lon": 103.945 },
    "Phanom Dong Rak": { "province": "Surin", "lat": 14.445556, "lon": 103.304444 },
    "Si Narong": { "province": "Surin", "lat": 14.76694, "lon": 103.87472 },
    "Khwao Sinarin": { "province": "Surin", "lat": 15.00222, "lon": 103.59639 },
    "Non Narai": { "province": "Surin", "lat": 15.23833, "lon": 103.91306 },
    "Mueang Si Sa Ket": { "province": "Si Sa Ket", "lat": 15.12056, "lon": 104.32222 },
    "Yang Chum Noi": { "province": "Si Sa Ket", "lat": 15.26167, "lon": 104.40167 },
    "Kanthararom": { "province": "Si Sa Ket", "lat": 15.10667, "lon": 104.57 },
    "Kantharalak": { "province": "Si Sa Ket", "lat": 14.64, "lon": 104.65 },
    "Khukhan": { "province": "Si Sa Ket", "lat": 14.71333, "lon": 104.19833 },
    "Phrai Bueng": { "province": "Si Sa Ket", "lat": 14.74833, "lon": 104.36167 },
    "Prang Ku": { "province": "Si Sa Ket", "lat": 14.85667, "lon": 104.04 },
    "Khun Han": { "province": "Si Sa Ket", "lat": 14.61667, "lon": 104.425 },
    "Rasi Salai": { "province": "Si Sa Ket", "lat": 15.3602599, "lon": 104.2158563 },
    "Uthumphon Phisai": { "province": "Si Sa Ket", "lat": 15.0769589, "lon": 104.0922843 },
    "Bueng Bun": { "province": "Si Sa Ket", "lat": 15.3303, "lon": 104.0449 },
    "Huai Thap Than": { "province": "Si Sa Ket", "lat": 15.0274252, "lon": 104.0762344 },
    "Non Khun": { "province": "Si Sa Ket", "lat": 14.92833, "lon": 104.69 },
    "Si Rattana": { "province": "Si Sa Ket", "lat": 14.79333, "lon": 104.47 },
    "Nam Kliang": { "province": "Si Sa Ket", "lat": 14.92833, "lon": 104.51333 },
    "Wang Hin": { "province": "Si Sa Ket", "lat": 14.94833, "lon": 104.23 },
    "Phu Sing": { "province": "Si Sa Ket", "lat": 14.54667, "lon": 104.131667 },
    "Mueang Chan": { "province": "Si Sa Ket", "lat": 15.19167, "lon": 104.05333 },
    "Benchalak": { "province": "Si Sa Ket", "lat": 14.795, "lon": 104.67 },
    "Phayu": { "province": "Si Sa Ket", "lat": 14.90333, "lon": 104.39167 },
    "Pho Si Suwan": { "province": "Si Sa Ket", "lat": 15.23167, "lon": 104.07667 },
    "Sila Lat": { "province": "Si Sa Ket", "lat": 15.47333, "lon": 104.07167 },
    "Mueang Ubon Ratchathani": { "province": "Ubon Ratchathani", "lat": 15.23844, "lon": 104.84866 },
    "Si Mueang Mai": { "province": "Ubon Ratchathani", "lat": 15.483, "lon": 105.283 },
    "Khong Chiam": { "province": "Ubon Ratchathani", "lat": 15.45432, "lon": 105.50363 },
    "Khueang Nai": { "province": "Ubon Ratchathani", "lat": 15.39097, "lon": 104.54363 },
    "Khemarat": { "province": "Ubon Ratchathani", "lat": 15.97114, "lon": 105.15168 },
    "Det Udom": { "province": "Ubon Ratchathani", "lat": 14.90598, "lon": 105.07836 },
    "Na Chaluai": { "province": "Ubon Ratchathani", "lat": 14.56695, "lon": 105.22367 },
    "Nam Yuen": { "province": "Ubon Ratchathani", "lat": 14.488333, "lon": 105 },
    "Buntharik": { "province": "Ubon Ratchathani", "lat": 14.70495, "lon": 105.40444 },
    "Trakan Phuet Phon": { "province": "Ubon Ratchathani", "lat": 15.6136, "lon": 105.06444 },
    "Kut Khaopun": { "province": "Ubon Ratchathani", "lat": 15.81907, "lon": 105.04066 },
    "Muang Sam Sip": { "province": "Ubon Ratchathani", "lat": 15.517, "lon": 104.733 },
    "Warin Chamrap": { "province": "Ubon Ratchathani", "lat": 15.19319, "lon": 104.8628 },
    "Phibun Mangsahan": { "province": "Ubon Ratchathani", "lat": 15.24467, "lon": 105.22908 },
    "Tan Sum": { "province": "Ubon Ratchathani", "lat": 15.39035, "lon": 105.16273 },
    "Pho Sai": { "province": "Ubon Ratchathani", "lat": 15.76739, "lon": 105.34339 },
    "Samrong": { "province": "Ubon Ratchathani", "lat": 14.98532, "lon": 104.79139 },
    "Don Mot Daeng": { "province": "Ubon Ratchathani", "lat": 15.37694, "lon": 105.02107 },
    "Sirindhorn": { "province": "Ubon Ratchathani", "lat": 15.11249, "lon": 105.42837 },
    "Thung Si Udom": { "province": "Ubon Ratchathani", "lat": 14.73303, "lon": 104.90897 },
    "Na Yia": { "province": "Ubon Ratchathani", "lat": 15.0414, "lon": 105.05729 },
    "Na Tan": { "province": "Ubon Ratchathani", "lat": 15.8991237, "lon": 105.3014109 },
    "Lao Suea Kok": { "province": "Ubon Ratchathani", "lat": 15.43952, "lon": 104.91662 },
    "Sawang Wirawong": { "province": "Ubon Ratchathani", "lat": 15.23439, "lon": 105.076 },
    "Nam Khun": { "province": "Ubon Ratchathani", "lat": 14.55508, "lon": 104.89872 },
    "Mueang Yasothon": { "province": "Yasothon", "lat": 15.798429, "lon": 104.148913 },
    "Sai Mun": { "province": "Yasothon", "lat": 15.948604, "lon": 104.198638 },
    "Kut Chum": { "province": "Yasothon", "lat": 16.03167, "lon": 104.34 },
    "Kham Khuean Kaeo": { "province": "Yasothon", "lat": 15.64488448, "lon": 104.2541787 },
    "Pa Tio": { "province": "Yasothon", "lat": 15.83139, "lon": 104.38556 },
    "Maha Chana Chai": { "province": "Yasothon", "lat": 15.53167, "lon": 104.24167 },
    "Kho Wang": { "province": "Yasothon", "lat": 15.3678978, "lon": 104.3576377 },
    "Loeng Nok Tha": { "province": "Yasothon", "lat": 16.20778, "lon": 104.55472 },
    "Thai Charoen": { "province": "Yasothon", "lat": 16.063056, "lon": 104.447222 },
    "Mueang Chaiyaphum": { "province": "Chaiyaphum", "lat": 15.8676, "lon": 102.0261 },
    "Ban Khwao": { "province": "Chaiyaphum", "lat": 15.8097, "lon": 101.8208 },
    "Khon Sawan": { "province": "Chaiyaphum", "lat": 15.9443, "lon": 102.3006 },
    "Kaset Sombun": { "province": "Chaiyaphum", "lat": 16.3352, "lon": 101.9348 },
    "Nong Bua Daeng": { "province": "Chaiyaphum", "lat": 16.1215, "lon": 101.6158 },
    "Chatturat": { "province": "Chaiyaphum", "lat": 15.6007, "lon": 101.8436 },
    "Bamnet Narong": { "province": "Chaiyaphum", "lat": 15.4502, "lon": 101.6385 },
    "Nong Bua Rawe": { "province": "Chaiyaphum", "lat": 15.7953, "lon": 101.6613 },
    "Thep Sathit": { "province": "Chaiyaphum", "lat": 15.7126, "lon": 101.4793 },
    "Phu Khiao": { "province": "Chaiyaphum", "lat": 16.3194, "lon": 102.209 },
    "Ban Thaen": { "province": "Chaiyaphum", "lat": 16.38167, "lon": 102.35678 },
    "Kaeng Khro": { "province": "Chaiyaphum", "lat": 16.10864, "lon": 102.25808 },
    "Khon San": { "province": "Chaiyaphum", "lat": 16.49605084, "lon": 102.0494713 },
    "Phakdi Chumphon": { "province": "Chaiyaphum", "lat": 15.905, "lon": 101.421666 },
    "Noen Sa-nga": { "province": "Chaiyaphum", "lat": 15.55169, "lon": 101.99408 },
    "Sap Yai": { "province": "Chaiyaphum", "lat": 15.6062, "lon": 101.65005 },
    "Mueang Amnat Charoen": { "province": "Amnat Charoen", "lat": 15.85851, "lon": 104.62883 },
    "Chanuman": { "province": "Amnat Charoen", "lat": 16.12553, "lon": 104.92279 },
    "Pathum Ratchawongsa": { "province": "Amnat Charoen", "lat": 15.89542, "lon": 104.89611 },
    "Phana": { "province": "Amnat Charoen", "lat": 15.66895, "lon": 104.85675 },
    "Senangkhanikhom": { "province": "Amnat Charoen", "lat": 16.0513, "lon": 104.68441 },
    "Hua Taphan": { "province": "Amnat Charoen", "lat": 15.67594, "lon": 104.52702 },
    "Lue Amnat": { "province": "Amnat Charoen", "lat": 15.70781, "lon": 104.71118 },
    "Mueang Bueng Kan": { "province": "Bueng Kan", "lat": 18.2995, "lon": 103.64016 },
    "Phon Charoen": { "province": "Bueng Kan", "lat": 18.033038, "lon": 103.708356 },
    "So Phisai": { "province": "Bueng Kan", "lat": 18.14176, "lon": 103.44923 },
    "Seka": { "province": "Bueng Kan", "lat": 17.989767, "lon": 103.937599 },
    "Pak Khat": { "province": "Bueng Kan", "lat": 18.27738, "lon": 103.3414 },
    "Bueng Khong Long": { "province": "Bueng Kan", "lat": 18.00899, "lon": 104.07302 },
    "Si Wilai": { "province": "Bueng Kan", "lat": 18.2244764, "lon": 103.7405019 },
    "Bung Khla": { "province": "Bueng Kan", "lat": 18.24726, "lon": 103.99427 },
    "Mueang Nong Bua Lam Phu": { "province": "Nong Bua Lam Phu", "lat": 17.20406, "lon": 102.44068 },
    "Na Klang": { "province": "Nong Bua Lam Phu", "lat": 17.30722, "lon": 102.18861 },
    "Non Sang": { "province": "Nong Bua Lam Phu", "lat": 16.8687, "lon": 102.56642 },
    "Si Bun Rueang": { "province": "Nong Bua Lam Phu", "lat": 16.96705, "lon": 102.27607 },
    "Suwannakhuha": { "province": "Nong Bua Lam Phu", "lat": 17.54605, "lon": 102.24852 },
    "Na Wang": { "province": "Nong Bua Lam Phu", "lat": 17.3429, "lon": 102.07157 },
    "Mueang Khon Kaen": { "province": "Khon Kaen", "lat": 16.439625, "lon": 102.828728 },
    "Ban Fang": { "province": "Khon Kaen", "lat": 16.49162, "lon": 102.61669 },
    "Phra Yuen": { "province": "Khon Kaen", "lat": 16.30823, "lon": 102.67108 },
    "Nong Ruea": { "province": "Khon Kaen", "lat": 16.49166, "lon": 102.44572 },
    "Chum Phae": { "province": "Khon Kaen", "lat": 16.61591, "lon": 102.09619 },
    "Si Chomphu": { "province": "Khon Kaen", "lat": 16.8, "lon": 102.2 },
    "Nam Phong": { "province": "Khon Kaen", "lat": 16.7, "lon": 102.867 },
    "Ubolratana": { "province": "Khon Kaen", "lat": 16.77766, "lon": 102.67536 },
    "Kranuan": { "province": "Khon Kaen", "lat": 16.70672, "lon": 103.07878 },
    "Ban Phai": { "province": "Khon Kaen", "lat": 16.05997, "lon": 102.73097 },
    "Pueai Noi": { "province": "Khon Kaen", "lat": 15.86978, "lon": 102.90767 },
    "Phon": { "province": "Khon Kaen", "lat": 15.816, "lon": 102.59981 },
    "Waeng Yai": { "province": "Khon Kaen", "lat": 15.92532, "lon": 102.46221 },
    "Waeng Noi": { "province": "Khon Kaen", "lat": 15.80631, "lon": 102.42358 },
    "Nong Song Hong": { "province": "Khon Kaen", "lat": 15.76419, "lon": 102.79174 },
    "Phu Wiang": { "province": "Khon Kaen", "lat": 16.6791, "lon": 102.39893 },
    "Mancha Khiri": { "province": "Khon Kaen", "lat": 16.117, "lon": 102.55 },
    "Chonnabot": { "province": "Khon Kaen", "lat": 16.02249, "lon": 102.55946 },
    "Khao Suan Kwang": { "province": "Khon Kaen", "lat": 16.91682, "lon": 102.78206 },
    "Phu Pha Man": { "province": "Khon Kaen", "lat": 16.71864, "lon": 101.87081 },
    "Sam Sung": { "province": "Khon Kaen", "lat": 16.54306, "lon": 103.07972 },
    "Khok Pho Chai": { "province": "Khon Kaen", "lat": 16.07194, "lon": 102.38428 },
    "Nong Na Kham": { "province": "Khon Kaen", "lat": 16.8043, "lon": 102.33647 },
    "Ban Haet": { "province": "Khon Kaen", "lat": 16.20067, "lon": 102.77465 },
    "Non Sila": { "province": "Khon Kaen", "lat": 15.9642, "lon": 102.69074 },
    "Wiang Kao": { "province": "Khon Kaen", "lat": 15.9642, "lon": 102.69074 },
    "Mueang Udon Thani": { "province": "Udon Thani", "lat": 17.415, "lon": 102.786667 },
    "Kut Chap": { "province": "Udon Thani", "lat": 17.4255, "lon": 102.5666 },
    "Nong Wua So": { "province": "Udon Thani", "lat": 17.1625, "lon": 102.5727 },
    "Kumphawapi": { "province": "Udon Thani", "lat": 17.1095, "lon": 103.0069 },
    "Non Sa-at": { "province": "Udon Thani", "lat": 16.9225, "lon": 102.9436 },
    "Nong Han": { "province": "Udon Thani", "lat": 17.3605, "lon": 103.1055 },
    "Thung Fon": { "province": "Udon Thani", "lat": 17.4852, "lon": 103.1971 },
    "Chai Wan": { "province": "Udon Thani", "lat": 17.2866, "lon": 103.2233 },
    "Si That": { "province": "Udon Thani", "lat": 16.9727, "lon": 103.2166 },
    "Wang Sam Mo": { "province": "Udon Thani", "lat": 16.9497, "lon": 103.4286 },
    "Ban Dung": { "province": "Udon Thani", "lat": 17.7388, "lon": 103.2202 },
    "Ban Phue": { "province": "Udon Thani", "lat": 17.6875, "lon": 102.4727 },
    "Nam Som": { "province": "Udon Thani", "lat": 17.7705, "lon": 102.1897 },
    "Phen": { "province": "Udon Thani", "lat": 17.6646, "lon": 102.9436 },
    "Sang Khom": { "province": "Udon Thani", "lat": 17.8255, "lon": 103.0886 },
    "Nong Saeng": { "province": "Udon Thani", "lat": 17.1675, "lon": 102.7805 },
    "Na Yung": { "province": "Udon Thani", "lat": 17.9041, "lon": 102.1175 },
    "Phibun Rak": { "province": "Udon Thani", "lat": 17.5491, "lon": 103.0575 },
    "Ku Kaeo": { "province": "Udon Thani", "lat": 17.1722, "lon": 103.153 },
    "Prachaksinlapakhom": { "province": "Udon Thani", "lat": 17.2547, "lon": 103.0011 },
    "Mueang Loei": { "province": "Loei", "lat": 17.5537, "lon": 101.7524 },
    "Na Duang": { "province": "Loei", "lat": 17.5371, "lon": 102.0261 },
    "Chiang Khan": { "province": "Loei", "lat": 17.8334, "lon": 101.7524 },
    "Pak Chom": { "province": "Loei", "lat": 18.0216, "lon": 101.8883 },
    "Dan Sai": { "province": "Loei", "lat": 17.2126, "lon": 101.207 },
    "Na Haeo": { "province": "Loei", "lat": 17.4091, "lon": 101.0259 },
    "Phu Ruea": { "province": "Loei", "lat": 17.3887, "lon": 101.3885 },
    "Tha Li": { "province": "Loei", "lat": 17.6631, "lon": 101.4793 },
    "Wang Saphung": { "province": "Loei", "lat": 17.2743, "lon": 101.7524 },
    "Phu Kradueng": { "province": "Loei", "lat": 16.897, "lon": 101.8436 },
    "Phu Luang": { "province": "Loei", "lat": 17.0936, "lon": 101.6613 },
    "Pha Khao": { "province": "Loei", "lat": 17.0502, "lon": 102.0033 },
    "Erawan": { "province": "Loei", "lat": 17.2854, "lon": 101.9576 },
    "Nong Hin": { "province": "Loei", "lat": 17.1075, "lon": 101.8208 },
    "Mueang Nong Khai": { "province": "Nong Khai", "lat": 17.8379, "lon": 102.7825 },
    "Tha Bo": { "province": "Nong Khai", "lat": 17.8064, "lon": 102.5528 },
    "So Phisai": { "province": "Nong Khai", "lat": 17.9309, "lon": 103.1279 },
    "Si Chiang Mai": { "province": "Nong Khai", "lat": 17.9382, "lon": 102.4954 },
    "Sangkhom": { "province": "Nong Khai", "lat": 18.0599, "lon": 102.2319 },
    "Sakhrai": { "province": "Nong Khai", "lat": 17.658, "lon": 102.6905 },
    "Fao Rai": { "province": "Nong Khai", "lat": 17.9895, "lon": 103.2894 },
    "Rattanawapi": { "province": "Nong Khai", "lat": 18.22, "lon": 103.18 },
    "Pho Tak": { "province": "Nong Khai", "lat": 17.9084, "lon": 102.4151 },
    "Mueang Maha Sarakham": { "province": "Maha Sarakham", "lat": 16.18939534, "lon": 103.2975673 },
    "Kae Dam": { "province": "Maha Sarakham", "lat": 16.04961, "lon": 103.38953 },
    "Kosum Phisai": { "province": "Maha Sarakham", "lat": 16.23882, "lon": 102.99081 },
    "Kantharawichai": { "province": "Maha Sarakham", "lat": 16.317, "lon": 103.3 },
    "Chiang Yuen": { "province": "Maha Sarakham", "lat": 16.40942, "lon": 103.07005 },
    "Borabue": { "province": "Maha Sarakham", "lat": 15.98881, "lon": 103.12521 },
    "Na Chueak": { "province": "Maha Sarakham", "lat": 15.8018, "lon": 103.05725 },
    "Phayakkhaphum Phisai": { "province": "Maha Sarakham", "lat": 15.51963, "lon": 103.23268 },
    "Wapi Pathum": { "province": "Maha Sarakham", "lat": 15.84523, "lon": 103.37678 },
    "Na Dun": { "province": "Maha Sarakham", "lat": 15.733, "lon": 103.233 },
    "Yang Sisurat": { "province": "Maha Sarakham", "lat": 15.64953, "lon": 103.08916 },
    "Kut Rang": { "province": "Maha Sarakham", "lat": 16.1, "lon": 103.017 },
    "Chuen Chom": { "province": "Maha Sarakham", "lat": 16.54801, "lon": 103.14796 },
    "Mueang Roi Et": { "province": "Roi Et", "lat": 16.01497, "lon": 103.61584 },
    "Kaset Wisai": { "province": "Roi Et", "lat": 15.6581, "lon": 103.6131 },
    "Pathum Rat": { "province": "Roi Et", "lat": 15.62371, "lon": 103.38733 },
    "Chaturaphak Phiman": { "province": "Roi Et", "lat": 15.8447, "lon": 103.5668 },
    "Thawat Buri": { "province": "Roi Et", "lat": 16.111, "lon": 103.8042 },
    "Phanom Phrai": { "province": "Roi Et", "lat": 15.693, "lon": 104.0755 },
    "Phon Thong": { "province": "Roi Et", "lat": 16.29007, "lon": 103.96388 },
    "Pho Chai": { "province": "Roi Et", "lat": 16.32827, "lon": 103.77033 },
    "Nong Phok": { "province": "Roi Et", "lat": 16.30357, "lon": 104.22144 },
    "Selaphum": { "province": "Roi Et", "lat": 16.047, "lon": 104.00031 },
    "Suwannaphum": { "province": "Roi Et", "lat": 15.60348, "lon": 103.80207 },
    "Mueang Suang": { "province": "Roi Et", "lat": 15.78619, "lon": 103.75071 },
    "Phon Sai": { "province": "Roi Et", "lat": 15.49244, "lon": 103.94139 },
    "At Samat": { "province": "Roi Et", "lat": 15.84014, "lon": 103.86619 },
    "Moei Wadi": { "province": "Roi Et", "lat": 16.38944, "lon": 104.1572 },
    "Si Somdet": { "province": "Roi Et", "lat": 16.01947, "lon": 103.51427 },
    "Changhan": { "province": "Roi Et", "lat": 16.16213, "lon": 103.61605 },
    "Chiang Khwan": { "province": "Roi Et", "lat": 16.14887, "lon": 103.74924 },
    "Nong Hi": { "province": "Roi Et", "lat": 15.57983, "lon": 104.01134 },
    "Thung Khao Luang": { "province": "Roi Et", "lat": 15.98966, "lon": 103.87086},
    "Mueang Kalasin": { "province": "Kalasin", "lat": 16.50918, "lon": 103.54984 },
    "Na Mon": { "province": "Kalasin", "lat": 16.56994, "lon": 103.78015 },
    "Kamalasai": { "province": "Kalasin", "lat": 16.27999, "lon": 103.6001 },
    "Rong Kham": { "province": "Kalasin", "lat": 16.27925, "lon": 103.71238 },
    "Kuchinarai": { "province": "Kalasin", "lat": 16.51977, "lon": 104.05097 },
    "Khao Wong": { "province": "Kalasin", "lat": 16.70008, "lon": 104.09019 },
    "Yang Talat": { "province": "Kalasin", "lat": 16.39982, "lon": 103.36785 },
    "Huai Mek": { "province": "Kalasin", "lat": 16.58975, "lon": 103.23547 },
    "Sahatsakhan": { "province": "Kalasin", "lat": 16.70987, "lon": 103.57909 },
    "Kham Muang": { "province": "Kalasin", "lat": 16.93884, "lon": 103.6398 },
    "Tha Khantho": { "province": "Kalasin", "lat": 16.88333, "lon": 103.25 },
    "Nong Kung Si": { "province": "Kalasin", "lat": 16.65, "lon": 103.3 },
    "Somdet": { "province": "Kalasin", "lat": 16.704444, "lon": 103.75 },
    "Huai Phueng": { "province": "Kalasin", "lat": 16.6521, "lon": 103.8912 },
    "Sam Chai": { "province": "Kalasin", "lat": 16.86909, "lon": 103.52243 },
    "Na Khu": { "province": "Kalasin", "lat": 16.75018, "lon": 104.00062 },
    "Don Chan": { "province": "Kalasin", "lat": 16.46667, "lon": 103.71667 },
    "Khong Chai": { "province": "Kalasin", "lat": 16.25845, "lon": 103.48167 },
    "Mueang Sakon Nakhon": { "province": "Sakon Nakhon", "lat": 17.16163, "lon": 104.10519 },
    "Kusuman": { "province": "Sakon Nakhon", "lat": 17.35402, "lon": 104.26677 },
    "Kut Bak": { "province": "Sakon Nakhon", "lat": 17.0416, "lon": 103.7637 },
    "Phanna Nikhom": { "province": "Sakon Nakhon", "lat": 17.31369, "lon": 103.89791 },
    "Phang Khon": { "province": "Sakon Nakhon", "lat": 17.38267, "lon": 103.75146 },
    "Waritchaphum": { "province": "Sakon Nakhon", "lat": 17.25692, "lon": 103.62283 },
    "Nikhom Nam Un": { "province": "Sakon Nakhon", "lat": 17.18314, "lon": 103.71842 },
    "Wanon Niwat": { "province": "Sakon Nakhon", "lat": 17.61516, "lon": 103.76536 },
    "Kham Ta Kla": { "province": "Sakon Nakhon", "lat": 17.82547, "lon": 103.78598 },
    "Ban Muang": { "province": "Sakon Nakhon", "lat": 17.89569, "lon": 103.52793 },
    "Akat Amnuai": { "province": "Sakon Nakhon", "lat": 17.63589, "lon": 103.9759 },
    "Sawang Daen Din": { "province": "Sakon Nakhon", "lat": 17.4559, "lon": 103.4495 },
    "Song Dao": { "province": "Sakon Nakhon", "lat": 17.30883, "lon": 103.44987 },
    "Tao Ngoi": { "province": "Sakon Nakhon", "lat": 16.94022, "lon": 104.15863 },
    "Khok Si Suphan": { "province": "Sakon Nakhon", "lat": 17.02297, "lon": 104.30346 },
    "Charoen Sin": { "province": "Sakon Nakhon", "lat": 17.63327, "lon": 103.52218 },
    "Phon Na Kaeo": { "province": "Sakon Nakhon", "lat": 17.21248, "lon": 104.307 },
    "Phu Phan": { "province": "Sakon Nakhon", "lat": 16.92862, "lon": 103.924 },
    "Mueang Nakhon Phanom": { "province": "Nakhon Phanom", "lat": 17.31965, "lon": 104.68232 },
    "Pla Pak": { "province": "Nakhon Phanom", "lat": 17.218, "lon": 104.5306 },
    "Tha Uthen": { "province": "Nakhon Phanom", "lat": 17.5768, "lon": 104.5947 },
    "Ban Phaeng": { "province": "Nakhon Phanom", "lat": 17.86955, "lon": 104.22744 },
    "That Phanom": { "province": "Nakhon Phanom", "lat": 16.93636, "lon": 104.71039 },
    "Renu Nakhon": { "province": "Nakhon Phanom", "lat": 17.0508, "lon": 104.65664 },
    "Na Kae": { "province": "Nakhon Phanom", "lat": 16.95737, "lon": 104.49349 },
    "Si Songkhram": { "province": "Nakhon Phanom", "lat": 17.62989, "lon": 104.24495 },
    "Na Wa": { "province": "Nakhon Phanom", "lat": 17.4897, "lon": 104.10056 },
    "Phon Sawan": { "province": "Nakhon Phanom", "lat": 17.46868, "lon": 104.42234 },
    "Na Thom": { "province": "Nakhon Phanom", "lat": 17.82134, "lon": 104.10871 },
    "Wang Yang": { "province": "Nakhon Phanom", "lat": 17.05751, "lon": 104.45333 },
    "Mueang Mukdahan": { "province": "Mukdahan", "lat": 16.5511, "lon": 104.64605 },
    "Nikhom Kham Soi": { "province": "Mukdahan", "lat": 16.3717, "lon": 104.5515 },
    "Don Tan": { "province": "Mukdahan", "lat": 16.28796, "lon": 104.82231 },
    "Dong Luang": { "province": "Mukdahan", "lat": 16.828, "lon": 104.5073 },
    "Khamcha-i": { "province": "Mukdahan", "lat": 16.61452, "lon": 104.35929 },
    "Wan Yai": { "province": "Mukdahan", "lat": 16.7203, "lon": 104.72865 },
    "Nong Sung": { "province": "Mukdahan", "lat": 16.44759, "lon": 104.33649 },
    "Mueang Lamphun": { "province": "Lamphun", "lat": 18.578333, "lon": 99.018333 },
    "Pa Sang": { "province": "Lamphun", "lat": 18.525833, "lon": 98.939167 },
    "Ban Hong": { "province": "Lamphun", "lat": 18.331111, "lon": 98.818889 },
    "Mae Tha (Lamphun)": { "province": "Lamphun", "lat": 18.461944, "lon": 99.134722 },
    "Li": { "province": "Lamphun", "lat": 17.803333, "lon": 98.95 },
    "Thung Hua Chang": { "province": "Lamphun", "lat": 18.003056, "lon": 99.027778 },
    "Ban Thi": { "province": "Lamphun", "lat": 18.651667, "lon": 99.125 },
    "Wiang Nong Long": { "province": "Lamphun", "lat": 18.393333, "lon": 98.718333 },
    "Mueang Lampang": { "province": "Lampang", "lat": 18.29232, "lon": 99.49277 },
    "Ko Kha": { "province": "Lampang", "lat": 18.191667, "lon": 99.395556 },
    "Ngao": { "province": "Lampang", "lat": 18.7793, "lon": 99.9465 },
    "Chae Hom": { "province": "Lampang", "lat": 18.712778, "lon": 99.5575 },
    "Mae Mo": { "province": "Lampang", "lat": 18.275, "lon": 99.65 },
    "Mae Tha (Lampang)": { "province": "Lampang", "lat": 18.135, "lon": 99.513333 },
    "Sop Prap": { "province": "Lampang", "lat": 17.880833, "lon": 99.337222 },
    "Soem Ngam": { "province": "Lampang", "lat": 18.065278, "lon": 99.247222 },
    "Hang Chat": { "province": "Lampang", "lat": 18.321944, "lon": 99.317778 },
    "Wang Nuea": { "province": "Lampang", "lat": 19.0756, "lon": 99.6787 },
    "Thoen": { "province": "Lampang", "lat": 17.5455, "lon": 99.2786 },
    "Mae Phrik": { "province": "Lampang", "lat": 17.5089, "lon": 99.0571 },
    "Mueang Pan": { "province": "Lampang", "lat": 18.7824, "lon": 99.4339 },
    "Mueang Uttaradit": { "province": "Uttaradit", "lat": 17.6256, "lon": 100.0942 },
    "Tron": { "province": "Uttaradit", "lat": 17.47231, "lon": 100.13467 },
    "Tha Pla": { "province": "Uttaradit", "lat": 17.85868, "lon": 100.47061 },
    "Nam Pat": { "province": "Uttaradit", "lat": 17.72738, "lon": 100.72738 },
    "Fak Tha": { "province": "Uttaradit", "lat": 17.88885, "lon": 100.88885 },
    "Ban Khok": { "province": "Uttaradit", "lat": 18.0661, "lon": 101.0661 },
    "Phichai": { "province": "Uttaradit", "lat": 17.35, "lon": 100.05 },
    "Laplae": { "province": "Uttaradit", "lat": 17.65741, "lon": 100.01484 },
    "Thong Saen Khan": { "province": "Uttaradit", "lat": 17.5414537, "lon": 100.3048897 },
    "Mueang Phrae": { "province": "Phrae", "lat": 18.14589, "lon": 100.14103 },
    "Rong Kwang": { "province": "Phrae", "lat": 18.33903, "lon": 100.31736 },
    "Long": { "province": "Phrae", "lat": 18.075, "lon": 99.83167 },
    "Sung Men": { "province": "Phrae", "lat": 18.04356, "lon": 100.10312 },
    "Den Chai": { "province": "Phrae", "lat": 17.98372, "lon": 100.05217 },
    "Song": { "province": "Phrae", "lat": 18.5769, "lon": 100.2152 },
    "Wang Chin": { "province": "Phrae", "lat": 17.86667, "lon": 99.6499 },
    "Nong Muang Khai": { "province": "Phrae", "lat": 18.267, "lon": 100.2 },
    "Mueang Nan": { "province": "Nan", "lat": 18.78378, "lon": 100.77899 },
    "Tha Wang Pha": { "province": "Nan", "lat": 19.10963, "lon": 100.7548 },
    "Pua": { "province": "Nan", "lat": 19.175, "lon": 100.916 },
    "Ban Luang": { "province": "Nan", "lat": 18.88375, "lon": 100.46014 },
    "Song Khwae": { "province": "Nan", "lat": 19.40911, "lon": 100.66174 },
    "Chiang Klang": { "province": "Nan", "lat": 19.29378, "lon": 100.86169 },
    "Phu Phiang": { "province": "Nan", "lat": 18.7803, "lon": 100.87497 },
    "Wiang Sa": { "province": "Nan", "lat": 18.55088, "lon": 100.72754 },
    "Thung Chang": { "province": "Nan", "lat": 19.4167, "lon": 100.8833 },
    "Nan Noi": { "province": "Nan", "lat": 18.30269, "lon": 100.72393 },
    "Mae Charim": { "province": "Nan", "lat": 18.717, "lon": 101.033 },
     "Chaloem Phra Kiat": { "province": "Nan", "lat": 19.579167, "lon": 101.081111 },
    "Na Muen": { "province": "Nan", "lat": 18.189444, "lon": 100.658889 },
    "Santi Suk": { "province": "Nan", "lat": 18.913333, "lon": 100.941389 },
    "Bo Kluea": { "province": "Nan", "lat": 19.148889, "lon": 101.1575 },
    "Mueang Phayao": { "province": "Phayao", "lat": 19.191667, "lon": 99.878333 },
    "Dok Kham Tai": { "province": "Phayao", "lat": 19.1505, "lon": 100.036 },
    "Phu Kamyao": { "province": "Phayao", "lat": 19.271667, "lon": 99.97 },
    "Mae Chai": { "province": "Phayao", "lat": 19.3756, "lon": 99.7902 },
    "Chun": { "province": "Phayao", "lat": 19.3559, "lon": 100.1479 },
    "Chiang Kham": { "province": "Phayao", "lat": 19.5064, "lon": 100.3947 },
    "Pong": { "province": "Phayao", "lat": 19.2244, "lon": 100.3947 },
    "Phu Sang": { "province": "Phayao", "lat": 19.6253, "lon": 100.3722 },
    "Chiang Muan": { "province": "Phayao", "lat": 18.886667, "lon": 100.303333 },
    "Mueang Chiang Rai": { "province": "Chiang Rai", "lat": 19.908611, "lon": 99.8325 },
    "Wiang Chai": { "province": "Chiang Rai", "lat": 19.883056, "lon": 99.933056 },
    "Chiang Khong": { "province": "Chiang Rai", "lat": 20.26, "lon": 100.406667 },
    "Thoeng": { "province": "Chiang Rai", "lat": 19.685, "lon": 100.193333 },
    "Phan": { "province": "Chiang Rai", "lat": 19.553889, "lon": 99.740556 },
    "Pa Daet": { "province": "Chiang Rai", "lat": 19.503333, "lon": 99.993333 },
    "Mae Chan": { "province": "Chiang Rai", "lat": 20.146667, "lon": 99.853333 },
    "Chiang Saen": { "province": "Chiang Rai", "lat": 20.275, "lon": 100.088333 },
    "Mae Sai": { "province": "Chiang Rai", "lat": 20.428056, "lon": 99.883611 },
    "Mae Suai": { "province": "Chiang Rai", "lat": 19.656667, "lon": 99.541667 },
    "Wiang Pa Pao": { "province": "Chiang Rai", "lat": 19.3046, "lon": 99.44925 },
    "Phaya Mengrai": { "province": "Chiang Rai", "lat": 19.848333, "lon": 100.153333 },
    "Wiang Kaen": { "province": "Chiang Rai", "lat": 20.111667, "lon": 100.513333 },
    "Khun Tan": { "province": "Chiang Rai", "lat": 19.833333, "lon": 100.258333 },
    "Mae Fa Luang": { "province": "Chiang Rai", "lat": 20.266667, "lon": 99.8 },
    "Mae Lao": { "province": "Chiang Rai", "lat": 19.79, "lon": 99.699722 },
    "Wiang Chiang Rung": { "province": "Chiang Rai", "lat": 20.011667, "lon": 100.056667 },
    "Doi Luang": { "province": "Chiang Rai", "lat": 20.118333, "lon": 100.1 },
    "Mueang Mae Hong": { "province": "Mae Hong Son", "lat": 19.303889, "lon": 97.977222 },
    "Khun Yuam": { "province": "Mae Hong Son", "lat": 18.839444, "lon": 97.951389 },
    "Pai": { "province": "Mae Hong Son", "lat": 19.358611, "lon": 98.44 },
    "Mae Sariang": { "province": "Mae Hong Son", "lat": 18.159444, "lon": 97.933611 },
    "Mae La Noi": { "province": "Mae Hong Son", "lat": 18.384444, "lon": 97.936944 },
    "Sop Moei": { "province": "Mae Hong Son", "lat": 17.961944, "lon": 97.933333 },
    "Pang Mapha": { "province": "Mae Hong Son", "lat": 19.5225, "lon": 98.246111 }
}
# Function to fetch and process weather data
def get_weather_data(district='Mueang Chon Buri'):
    
    params = {
        "lat": districts[district]['lat'],
        "lon": districts[district]['lon'],
        "appid": API_KEY,
        "units": "metric"
    }
    try:
        # Make API request
        response = requests.get(WEATHER_ENDPOINT, params=params, timeout=5)
        response.raise_for_status()  # Raise an exception for bad status codes
        data = response.json()


        time.sleep(1) 
        # Convert timestamp to datetime
        # created_at = datetime.fromtimestamp(data['dt'])

        dt = datetime.now()
        thai_tz = pytz.timezone('Asia/Bangkok')
        created_at = dt.replace(tzinfo=thai_tz)

        timestamp = datetime.now()
        
        # Create dictionary with required fields
        weather_dict = {
            'timestamp': timestamp,
            'year': timestamp.year,
            'month': timestamp.month,
            'day': timestamp.day,
            'hour': timestamp.hour,
            'minute': timestamp.minute,
            'created_at': created_at,
            'district': district,
            'province':districts[district]['province'],
            'location': data['name'],
            'weather_main': data['weather'][0]['main'],
            'weather_description': data['weather'][0]['description'],
            'main.temp': data['main']['temp'],
            'main.temp_min': data['main']['temp_min'],
            'main.temp_max': data['main']['temp_max'],
            'main.feels_like': data['main']['feels_like'],
            'main.pressure': data['main']['pressure'],
            'main.humidity': data['main']['humidity'],
            'main.sea_level': data['main']['sea_level'],
            'main.grnd_level': data['main']['grnd_level'],
            'visibility': data['visibility'],
            'wind.speed': data['wind']['speed'],
            'wind.deg': data['wind']['deg']
        }
        
        # Create DataFrame
        # df = pd.DataFrame([weather_dict])
        
        # return df
        return weather_dict
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None
    except KeyError as e:
        print(f"Error processing data: Missing key {e}")
        return None
# Function to fetch and process weather data
def get_pollution_data(district='Mueang Chon Buri'):
    
    params = {
        "lat": districts[district]['lat'],
        "lon": districts[district]['lon'],
        "appid": API_KEY,
        "units": "metric"
    }
    try:
        # Make API request
        response = requests.get(POLLUTION_ENDPOINT, params=params)
        response.raise_for_status()  # Raise an exception for bad status codes
        data = response.json()

        time.sleep(1) 
        # Convert timestamp to datetime
        # created_at = datetime.fromtimestamp(data['dt'])

        dt = datetime.now()
        thai_tz = pytz.timezone('Asia/Bangkok')
        created_at = dt.replace(tzinfo=thai_tz)

        timestamp = datetime.now()
        
        # Create dictionary with required fields
        pollution_dict = {
            'timestamp': timestamp,
            'year': timestamp.year,
            'month': timestamp.month,
            'day': timestamp.day,
            'hour': timestamp.hour,
            'minute': timestamp.minute,
            'created_at': created_at,
            'province':districts[district]['province'],
            'longtitude': data['coord']['lon'],
            'latitude': data['coord']['lat'],
            'components_co': data['list'][0]['components']['co'],
            'components_no': data['list'][0]['components']['no'],
            'components_no2': data['list'][0]['components']['no2'],
            'components_o3': data['list'][0]['components']['o3'],
            'components_so2': data['list'][0]['components']['so2'],
            'components_pm2_5': data['list'][0]['components']['pm2_5'],
            'components_pm10': data['list'][0]['components']['pm10'],
            'components_nh3': data['list'][0]['components']['nh3']
           
        }
        
        # Create DataFrame
        # df = pd.DataFrame([weather_dict])
        
        # return df
        return pollution_dict
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None
    except KeyError as e:
        print(f"Error processing data: Missing key {e}")
        return None
        
def get_combined_data(district='Mueang Chon Buri'):
    weather = get_weather_data(district)
    pollution = get_pollution_data(district)

    if weather is not None and pollution is not None:
        combined = {**weather, **pollution}
        return combined
    else:
        return None

In [23]:
df = pd.DataFrame([get_combined_data(p) for p in districts.keys()])
df.info()
df.head()

Error processing data: Missing key 'visibility'


KeyboardInterrupt: 

In [ ]:
dt = datetime.now()
thai_tz = pytz.timezone('Asia/Bangkok')
dt = dt.replace(tzinfo=thai_tz)
print(dt) 

In [ ]:
import pandas as pd

# lakeFS credentials from your docker-compose.yml
ACCESS_KEY = "access_key"
SECRET_KEY = "secret_key"

# lakeFS endpoint (running locally)
lakefs_endpoint = "http://lakefs-dev:8000/"

# lakeFS repository, branch, and file path
repo = "weather"
branch = "main"
path = "weather.parquet"

# Construct the full lakeFS S3-compatible path
lakefs_s3_path = f"s3a://{repo}/{branch}/{path}"

# Configure storage_options for lakeFS (S3-compatible)
storage_options = {
    "key": ACCESS_KEY,
    "secret": SECRET_KEY,
    "client_kwargs": {
        "endpoint_url": lakefs_endpoint
    }
}

In [ ]:
pip install pyarrow

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install fastparquet

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install s3fs

Note: you may need to restart the kernel to use updated packages.


In [ ]:
df.to_parquet(
    lakefs_s3_path,
    storage_options=storage_options,
    partition_cols=['year','month','day','hour'],
    
)

In [ ]:
from datetime import datetime
import pytz

dt = datetime.now()
thai_tz = pytz.timezone('Asia/Bangkok')
dt = dt.replace(tzinfo=thai_tz)

df.to_parquet(
    lakefs_s3_path,
    storage_options=storage_options,
    partition_cols=['year','month','day','hour']
)

In [ ]:
print(dt)